##  Setup, Dependencies, and Data Loading

This section focuses on setting up the environment for QLoRA training, including installing necessary packages, authenticating with required services (like Weights & Biases and Hugging Face), and loading the HotpotQA dataset.

### 📦 Installing Required Packages

We begin by installing the Python libraries necessary for our QLoRA fine-tuning pipeline.

**Note**: Replace `[repository_url]` with the actual URL of the Git repository you want to clone. You can find repositories related to Claude on platforms like GitHub.

In [ ]:
# Install required packages for PyTorch 2.1 container
import subprocess
import sys

def install_package(package, description=""):
    """Install package with proper error handling"""
    try:
        # Check if already installed
        if package.split('==')[0] in ['transformers', 'peft', 'datasets', 'accelerate', 'bitsandbytes', 'wandb', 'evaluate']:
            __import__(package.split('==')[0])
            print(f"✅ {package} already available")
            return True
    except ImportError:
        pass

    try:
        print(f"📦 Installing {package}... {description}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", package])
        print(f"✅ {package} installed successfully")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to install {package}: {e}")
        return False

# Essential packages for QLoRA training (compatible with PyTorch 2.1.0)
packages = [
    ("transformers>=4.36.0", "Latest Transformers with Mistral support"),
    ("peft>=0.7.0", "Parameter-Efficient Fine-Tuning"),
    ("datasets>=2.15.0", "HuggingFace Datasets"),
    ("accelerate>=0.25.0", "Distributed training support"),
    ("bitsandbytes>=0.41.0", "4-bit quantization"),
    ("wandb", "Experiment tracking"),
    ("evaluate", "Model evaluation metrics"),
    ("scipy", "Scientific computing"),
    ("scikit-learn", "ML utilities"),
    ("pydantic", "data validation"),
]


print("\n🔧 Installing required packages for RTX A5000...")
failed_packages = []

for package, desc in packages:
    if not install_package(package, desc):
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️ Failed to install: {failed_packages}")
    print("Please install manually or check container permissions")
else:
    print("\n✅ All packages installed successfully!")

print("\n🎯 RTX A5000 Optimization Settings:")
print("   - Batch size: 2 (optimal for 24GB VRAM)")
print("   - Sequence length: 2048 (memory efficient)")
print("   - Gradient accumulation: 4 steps")
print("   - Mixed precision: BF16 (A5000 optimized)")
print("   - Estimated training time: 3-4 hours")
print("   - Estimated cost: $1.50 - $2.00")

print("\n✅ Ready for cost-effective QLoRA training!")
print("📝 Next: Run GPU detection cell to confirm 24GB VRAM")


🔧 Installing required packages for RTX A5000...
📦 Installing transformers>=4.36.0... Latest Transformers with Mistral support
✅ transformers>=4.36.0 installed successfully
📦 Installing peft>=0.7.0... Parameter-Efficient Fine-Tuning
✅ peft>=0.7.0 installed successfully
📦 Installing datasets>=2.15.0... HuggingFace Datasets
✅ datasets>=2.15.0 installed successfully
📦 Installing accelerate>=0.25.0... Distributed training support
✅ accelerate>=0.25.0 installed successfully
📦 Installing bitsandbytes>=0.41.0... 4-bit quantization
✅ bitsandbytes>=0.41.0 installed successfully
✅ wandb already available
📦 Installing evaluate... Model evaluation metrics
✅ evaluate installed successfully
📦 Installing scipy... Scientific computing
✅ scipy installed successfully
📦 Installing scikit-learn... ML utilities
✅ scikit-learn installed successfully
📦 Installing pydantic... data validation
✅ pydantic installed successfully

✅ All packages installed successfully!

🎯 RTX A5000 Optimization Settings:
   - Batc

###  Cloud Platform Imports

Importing necessary libraries and modules, ensuring compatibility with cloud environments like RunPod.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json
import os
import zipfile
import shutil
from pathlib import Path
import time
import gc
from typing import Dict, List, Optional, Tuple
import warnings
from pydantic import BaseModel, Field
warnings.filterwarnings('ignore')

# Core ML libraries (should work on cloud platforms)
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, TrainerCallback, TrainerState
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
import evaluate
import wandb

print("✅ All imports successful on cloud platform!")
print("🌩️ Using standard transformers + PEFT stack")
print("⚡ Ready for QLoRA training with pre-configured packages!")

✅ All imports successful on cloud platform!
🌩️ Using standard transformers + PEFT stack
⚡ Ready for QLoRA training with pre-configured packages!


###  GPU Configuration and Cost Analysis

Detecting the available GPU and setting optimized parameters for QLoRA training, along with a realistic cost analysis based on dataset size.

In [ ]:
# RTX A5000 GPU Configuration (24GB VRAM optimized for cost-effectiveness)
import torch
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎯 CUDA available: {torch.cuda.is_available()}")
MAX_SEQ_LENGTH = 2048
BATCH_SIZE = 2
GRAD_ACCUM_STEPS = 4


if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU: {device}")
    print(f"💾 VRAM: {vram_gb:.1f} GB")

    # RTX A5000 optimized settings
    if "A5000" in device or (vram_gb >= 20 and vram_gb <= 30):
        GPU_TYPE = "RTX_A5000"
        MAX_SEQ_LENGTH = 10000  # Optimal for 24GB VRAM
        BATCH_SIZE = 2         # Memory efficient
        GRAD_ACCUM_STEPS = 4   # Effective batch size = 8
        HOURLY_RATE = 0.50     # RTX A5000 RunPod price
        SPEED_TOKENS_PER_SEC = 60  # Realistic speed
        print("🏆 RTX A5000 detected - using optimized settings")

    elif "4090" in device or (vram_gb >= 20 and vram_gb < 26):
        GPU_TYPE = "RTX_4090"
        MAX_SEQ_LENGTH = 10000
        BATCH_SIZE = 2
        GRAD_ACCUM_STEPS = 4
        HOURLY_RATE = 0.34
        SPEED_TOKENS_PER_SEC = 50
        print("✅ RTX 4090 detected - using memory-optimized settings")

    elif "A100" in device or vram_gb >= 40:
        GPU_TYPE = "A100"
        MAX_SEQ_LENGTH = 10000  # Can handle longer sequences
        BATCH_SIZE = 4         # Larger batch
        GRAD_ACCUM_STEPS = 2   # Effective batch size = 8
        HOURLY_RATE = 1.19     # A100 80GB RunPod price
        SPEED_TOKENS_PER_SEC = 150  # Much faster
        print("🏆 A100 detected - using high-performance settings")

    else:
        GPU_TYPE = "Other"
        MAX_SEQ_LENGTH = 10000
        BATCH_SIZE = 1
        GRAD_ACCUM_STEPS = 8
        HOURLY_RATE = 0.50
        SPEED_TOKENS_PER_SEC = 30
        print("⚠️ Unknown GPU - using conservative settings")

    print(f"\n⚙️ GPU Configuration: {GPU_TYPE}")
    print(f"📏 Max Sequence Length: {MAX_SEQ_LENGTH} tokens")
    print(f"📦 Batch Size: {BATCH_SIZE} (effective: {BATCH_SIZE * GRAD_ACCUM_STEPS})")
    print(f"💰 Hourly Rate: ${HOURLY_RATE}/hr")
    print(f"⚡ Speed: {SPEED_TOKENS_PER_SEC} tokens/second")

    # REALISTIC cost analysis for different dataset sizes
    def calculate_training_cost(train_size, epochs=2):
        effective_batch_size = BATCH_SIZE * GRAD_ACCUM_STEPS
        steps_per_epoch = train_size // effective_batch_size
        total_steps = steps_per_epoch * epochs

        # Realistic time calculation based on token processing
        tokens_per_step = effective_batch_size * MAX_SEQ_LENGTH
        seconds_per_step = tokens_per_step / SPEED_TOKENS_PER_SEC
        total_hours = (total_steps * seconds_per_step) / 3600
        total_cost = total_hours * HOURLY_RATE

        return {
            'steps_per_epoch': steps_per_epoch,
            'total_steps': total_steps,
            'training_hours': total_hours,
            'total_cost': total_cost,
            'tokens_per_step': tokens_per_step,
            'seconds_per_step': seconds_per_step
        }

    print(f"\n📊 REALISTIC TRAINING ANALYSIS:")
    print("=" * 50)

    # Different dataset size options
    options = [
        (2000, "Cost-optimized subset"),
        (10000, "Balanced training"),
        (90347, "Full dataset (expensive!)")
    ]

    for train_size, description in options:
        analysis = calculate_training_cost(train_size)
        pct_of_full = (train_size / 90347) * 100 if train_size <= 90347 else 100

        print(f"\n🎯 {description}: {train_size:,} examples ({pct_of_full:.1f}% of full dataset)")
        print(f"   Steps per epoch: {analysis['steps_per_epoch']}")
        print(f"   Total steps: {analysis['total_steps']}")
        print(f"   Training time: {analysis['training_hours']:.1f} hours")
        print(f"   💰 Total cost: ${analysis['total_cost']:.2f}")

        if analysis['training_hours'] > 100:
            print(f"   ⚠️  Very expensive - consider subset for experimentation")
        elif analysis['training_hours'] > 20:
            print(f"   ⚖️  Moderate cost - good for serious experiments")
        else:
            print(f"   ✅ Reasonable cost for experimentation")

    # Memory utilization analysis
    base_model_vram = 12  # QLoRA Mistral-7B in 4-bit
    training_overhead = 6  # Optimizer states, gradients
    batch_vram = (BATCH_SIZE * MAX_SEQ_LENGTH * 0.002)  # Dynamic batch memory
    total_vram_needed = base_model_vram + training_overhead + batch_vram

    print(f"\n💾 MEMORY UTILIZATION:")
    print(f"   Base model (4-bit): {base_model_vram} GB")
    print(f"   Training overhead: {training_overhead} GB")
    print(f"   Batch processing: {batch_vram:.1f} GB")
    print(f"   Total required: {total_vram_needed:.1f} GB")
    print(f"   Available VRAM: {vram_gb:.1f} GB")
    print(f"   Safety headroom: {vram_gb - total_vram_needed:.1f} GB ({((vram_gb - total_vram_needed)/vram_gb)*100:.0f}%)")

    if GPU_TYPE == "RTX_A5000":
        print(f"\n🎯 RTX A5000 REALISTIC EXPECTATIONS:")
        print(f"   ✅ 2,048 token sequences (optimal for 24GB)")
        print(f"   ✅ 2×4=8 effective batch size for stable gradients")
        print(f"   ✅ Professional workstation GPU performance")
        print(f"   ⚠️  Training times are much longer than initially estimated!")
        print(f"   💡 Consider starting with 2K samples to test, then scale up")
        print(f"   💰 Budget ~$15-20 for 2K samples, $50+ for 10K samples")

else:
    print("❌ No CUDA GPU detected! This notebook requires GPU for training.")
    raise RuntimeError("GPU required for QLoRA training")

print(f"\n✅ Configuration set for {GPU_TYPE} with REALISTIC time estimates!")

🔥 PyTorch version: 2.8.0+cu126
🎯 CUDA available: True
🚀 GPU: NVIDIA L4
💾 VRAM: 22.2 GB
🏆 RTX A5000 detected - using optimized settings

⚙️ GPU Configuration: RTX_A5000
📏 Max Sequence Length: 10000 tokens
📦 Batch Size: 2 (effective: 8)
💰 Hourly Rate: $0.5/hr
⚡ Speed: 60 tokens/second

📊 REALISTIC TRAINING ANALYSIS:

🎯 Cost-optimized subset: 2,000 examples (2.2% of full dataset)
   Steps per epoch: 250
   Total steps: 500
   Training time: 185.2 hours
   💰 Total cost: $92.59
   ⚠️  Very expensive - consider subset for experimentation

🎯 Balanced training: 10,000 examples (11.1% of full dataset)
   Steps per epoch: 1250
   Total steps: 2500
   Training time: 925.9 hours
   💰 Total cost: $462.96
   ⚠️  Very expensive - consider subset for experimentation

🎯 Full dataset (expensive!): 90,347 examples (100.0% of full dataset)
   Steps per epoch: 11293
   Total steps: 22586
   Training time: 8365.2 hours
   💰 Total cost: $4182.59
   ⚠️  Very expensive - consider subset for experimentation

💾 

###  Service Authentication

Setting up environment variables for Weights & Biases (W&B) and Hugging Face for experiment tracking and model access.

In [ ]:
import os

# Set W&B environment variables
# Replace with your actual W&B API Key
os.environ["WANDB_API_KEY"] = "YOUR_WANDB_KEY_HERE"
os.environ["WANDB_ENTITY"] = "jeffgong11235"  # Replace with your W&B entity
os.environ["WANDB_PROJECT"] = "hotpotqa-qlora"
os.environ["WANDB_RUN_GROUP"] = "deep-learning-rag"

# Set Hugging Face environment variables
# Replace with your actual Hugging Face Token (if needed for private models)
os.environ["HF_TOKEN"] = "hf_your_token_here"

print("✅ Environment variables set for W&B and Hugging Face")

✅ Environment variables set for W&B and Hugging Face


###  Initialize Weights & Biases

Logging into Weights & Biases and initializing a new run for tracking the training process.

In [ ]:
# W&B Configuration
if 'GPU_TYPE' not in globals():
  GPU_TYPE = 'CPU'
if 'MAX_SEQ_LENGTH' not in globals():
  MAX_SEQ_LENGTH = 1024
if 'BATCH_SIZE' not in globals():
  BATCH_SIZE = 1
if 'GRAD_ACCUM_STEPS' not in globals():
  GRAD_ACCUM_STEPS = 8
WANDB_ENTITY = "jeffgong11235"  # Replace with your W&B entity
WANDB_PROJECT = "hotpotqa-qlora"
RUN_NAME = f"mistral-7b-qlora-{GPU_TYPE.lower()}-{int(time.time())}"
GROUP = "deep-learning-rag"

print(f"🔧 W&B Configuration:")
print(f"   Entity: {WANDB_ENTITY}")
print(f"   Project: {WANDB_PROJECT}")
print(f"   Run Name: {RUN_NAME}")
print(f"   Group: {GROUP}")

# Login to W&B
print("\n🔐 Logging into Weights & Biases...")
wandb.login(key = "Your key here")  # Replace with your actual W&B API key

# Initialize W&B run
run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    name=RUN_NAME,
    group=GROUP,
    config={
        "base_model": "mistralai/Mistral-7B-Instruct-v0.2",
        "gpu_type": GPU_TYPE,
        "max_seq_length": MAX_SEQ_LENGTH,
        "batch_size": BATCH_SIZE,
        "grad_accum_steps": GRAD_ACCUM_STEPS,
        "lora_rank": 16,
        "lora_alpha": 32,
        "learning_rate": 5e-4,
        "epochs": 2,
        "quantization": "4bit-nf4"
    }
)

print(f"✅ W&B initialized! Run URL: {run.url}")

🔧 W&B Configuration:
   Entity: jeffgong11235
   Project: hotpotqa-qlora
   Run Name: mistral-7b-qlora-rtx_a5000-1760019526
   Group: deep-learning-rag

🔐 Logging into Weights & Biases...


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jeffgong11235 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ W&B initialized! Run URL: https://wandb.ai/jeffgong11235/hotpotqa-qlora/runs/ruvp840x


###  Hugging face Authentication

Logging into Hugging face for getting permission to use Hugging face models.

In [ ]:
# Log in to Hugging Face
from huggingface_hub import login
import os

# It's recommended to store your HF token securely in Colab Secrets
# and access it using userdata.get('HF_TOKEN')
# For this example, we'll use the environment variable set in the previous cell.

hf_token = os.environ.get("HF_TOKEN")

if hf_token:
    try:
        login(token=hf_token)
        print("✅ Successfully logged in to Hugging Face!")
    except Exception as e:
        print(f"❌ Failed to log in to Hugging Face: {e}")
        print("   Please ensure your HF_TOKEN environment variable is set correctly.")
else:
    print("⚠️ HF_TOKEN environment variable not found. Skipping Hugging Face login.")
    print("   Some models may require authentication. Please set HF_TOKEN in environment variables or Colab Secrets.")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Successfully logged in to Hugging Face!


### Load and Investigate Dataset

Loading the HotpotQA dataset and performing an initial investigation of its structure to understand how to process it for training.

In [ ]:
# Complete HotpotQA Structure Investigation
print("🔍 HOTPOTQA DATASET STRUCTURE INVESTIGATION")
print("=" * 60)

if 'HOURLY_RATE' not in globals():
  HOURLY_RATE = 0.50
# Load dataset
print("Loading HotpotQA dataset...")
dataset = load_dataset('hotpotqa/hotpot_qa', 'distractor')
train_data = dataset['train']
validation_data = dataset['validation']
print(f"✅ Dataset loaded: {len(train_data)} training examples")
print(f"✅ Dataset loaded: {len(validation_data)} validation examples")

# Get first example for detailed analysis
sample = train_data[0]

print(f"\n📋 COMPLETE SAMPLE STRUCTURE:")
print("=" * 60)

# Analyze each field systematically
for key, value in sample.items():
    print(f"\n🔍 FIELD: {key}")
    print(f"   Type: {type(value).__name__}")

    if hasattr(value, '__len__'):
        try:
            print(f"   Length: {len(value)}")
        except:
            pass

    # Special detailed handling for complex fields
    if key == 'context':
        print(f"   Raw value type: {type(value)}")
        print(f"   Is dict: {isinstance(value, dict)}")

        if isinstance(value, dict):
            print(f"   Dict keys: {list(value.keys())}")
            for dict_key, dict_value in value.items():
                print(f"   Key '{dict_key}': {type(dict_value).__name__}, Length: {len(dict_value) if hasattr(dict_value, '__len__') else 'N/A'}")
                if hasattr(dict_value, '__len__') and len(dict_value) > 0:
                    print(f"     First item: {type(dict_value[0]).__name__} - {repr(dict_value[0])}")

    elif key == 'supporting_facts':
        print(f"   Raw value type: {type(value)}")

        if isinstance(value, dict):
            print(f"   Dict keys: {list(value.keys())}")
            for dict_key, dict_value in value.items():
                print(f"   Key '{dict_key}': {type(dict_value).__name__}, Length: {len(dict_value) if hasattr(dict_value, '__len__') else 'N/A'}")
                if hasattr(dict_value, '__len__') and len(dict_value) > 0:
                    print(f"     First few items: {dict_value[:3]}")

    else:
        # For simple fields
        if isinstance(value, str) and len(value) > 100:
            print(f"   Value: {repr(value[:100])}...")
        else:
            print(f"   Value: {repr(value)}")

print(f"\n🧪 PRACTICAL ACCESS TESTS:")
print("=" * 60)

# Test actual processing patterns
context = sample['context']
supporting_facts = sample['supporting_facts']

print(f"Testing context processing:")
print(f"  Context type: {type(context)}")
if isinstance(context, dict):
    print(f"  Context keys: {list(context.keys())}")
    if 'title' in context and 'sentences' in context:
        titles = context['title']
        sentences = context['sentences']
        print(f"  Titles: {type(titles)}, Length: {len(titles)}")
        print(f"  Sentences: {type(sentences)}, Length: {len(sentences)}")
        print(f"  First title: {titles[0] if len(titles) > 0 else 'None'}")
        print(f"  First sentences: {sentences[0] if len(sentences) > 0 else 'None'}")

print(f"\nTesting supporting_facts processing:")
print(f"  Supporting facts type: {type(supporting_facts)}")
if isinstance(supporting_facts, dict):
    print(f"  Supporting facts keys: {list(supporting_facts.keys())}")
    if 'title' in supporting_facts and 'sent_id' in supporting_facts:
        titles = supporting_facts['title']
        sent_ids = supporting_facts['sent_id']
        print(f"  Titles: {titles}")
        print(f"  Sentence IDs: {sent_ids}")

# Dataset size configuration - FIXED SPEED_FACTOR issue
print(f"\n📊 DATASET SIZE CONFIGURATION:")
print("=" * 50)

# GPU-optimized subset for training
if 'GPU_TYPE' in globals():
    # Define SPEED_FACTOR based on GPU type
    if GPU_TYPE == "RTX_A5000":
        SPEED_FACTOR = 1.0
        TRAIN_SIZE = 2000   # Cost: ~$2.00, Time: 4 hours
        VAL_SIZE = 400
        print(f"🎯 RTX A5000 optimization: Using {TRAIN_SIZE} train, {VAL_SIZE} val samples")

    elif GPU_TYPE == "RTX_4090":
        SPEED_FACTOR = 0.8
        TRAIN_SIZE = 2000
        VAL_SIZE = 400
        print(f"🎯 RTX 4090 optimization: Using {TRAIN_SIZE} train, {VAL_SIZE} val samples")
    else:
        SPEED_FACTOR = 0.5
        TRAIN_SIZE = 1000
        VAL_SIZE = 200
        print(f"🎯 Conservative: Using {TRAIN_SIZE} train, {VAL_SIZE} val samples")

    # Cost analysis - FIXED with SPEED_FACTOR defined
    steps_per_epoch = TRAIN_SIZE // (BATCH_SIZE * GRAD_ACCUM_STEPS)
    total_steps = steps_per_epoch * 2  # 2 epochs
    training_hours = total_steps / (100 * SPEED_FACTOR)  # 100 steps/hour baseline with speed factor
    total_cost = training_hours * HOURLY_RATE

    print(f"\n💰 COST ANALYSIS:")
    print(f"   Training samples: {TRAIN_SIZE:,} ({TRAIN_SIZE/len(train_data)*100:.1f}% of full dataset)")
    print(f"   Steps per epoch: {steps_per_epoch}")
    print(f"   Total steps: {total_steps}")
    print(f"   Estimated time: {training_hours:.1f} hours")
    print(f"   Estimated cost: ${total_cost:.2f}")

    if TRAIN_SIZE < 5000:
        print(f"   💡 Using subset for cost optimization")
    elif TRAIN_SIZE < len(train_data):
        print(f"   ⚖️ Using partial dataset for balance of cost vs quality")
    else:
        print(f"   🏆 Using full dataset for maximum quality")

    train_sample = train_data.shuffle(seed=42).select(range(min(TRAIN_SIZE, len(train_data))))
    val_sample = validation_data.shuffle(seed=42).select(range(min(VAL_SIZE, len(validation_data))))
    print(f"✅ Working with: {len(train_sample)} train, {len(val_sample)} validation")
else:
    # Fallback if GPU_TYPE not defined - FIXED with SPEED_FACTOR
    SPEED_FACTOR = 0.5
    TRAIN_SIZE = 2000
    VAL_SIZE = 400
    train_sample = train_data.shuffle(seed=42).select(range(TRAIN_SIZE))
    val_sample = validation_data.shuffle(seed=42).select(range(VAL_SIZE))
    print(f"✅ Working with: {len(train_sample)} train, {len(val_sample)} validation")

print(f"\n🔧 STRUCTURE ANALYSIS COMPLETE!")
print(f"📋 Key findings:")
print(f"   - Context is a dict with 'title' and 'sentences' keys")
print(f"   - Supporting facts is a dict with 'title' and 'sent_id' keys")
print(f"   - Processing function needs to handle dict structure, not list structure")

🔍 HOTPOTQA DATASET STRUCTURE INVESTIGATION
Loading HotpotQA dataset...


README.md: 0.00B [00:00, ?B/s]

distractor/train-00000-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

distractor/train-00001-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

distractor/validation-00000-of-00001.par(…):   0%|          | 0.00/27.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

✅ Dataset loaded: 90447 training examples
✅ Dataset loaded: 7405 validation examples

📋 COMPLETE SAMPLE STRUCTURE:

🔍 FIELD: id
   Type: str
   Length: 24
   Value: '5a7a06935542990198eaf050'

🔍 FIELD: question
   Type: str
   Length: 70
   Value: "Which magazine was started first Arthur's Magazine or First for Women?"

🔍 FIELD: answer
   Type: str
   Length: 17
   Value: "Arthur's Magazine"

🔍 FIELD: type
   Type: str
   Length: 10
   Value: 'comparison'

🔍 FIELD: level
   Type: str
   Length: 6
   Value: 'medium'

🔍 FIELD: supporting_facts
   Type: dict
   Length: 2
   Raw value type: <class 'dict'>
   Dict keys: ['title', 'sent_id']
   Key 'title': list, Length: 2
     First few items: ["Arthur's Magazine", 'First for Women']
   Key 'sent_id': list, Length: 2
     First few items: [0, 0]

🔍 FIELD: context
   Type: dict
   Length: 2
   Raw value type: <class 'dict'>
   Is dict: True
   Dict keys: ['title', 'sentences']
   Key 'title': list, Length: 10
     First item: str - 'Radio Ci

# Data processing, COT prompt preparation, code implementation for RAG on prompt-generation, evaluation,

### Print 1 data points from train_sample and use the for creating chain of thought prompt

In [ ]:
# Print 8 data points from train_sample and use the for creating chain of thought prompt
print("Displaying 1 data points from train_sample:")
print("=" * 60)

# Ensure train_sample is available
if 'train_sample' in globals():
    num_examples_to_print = min(1, len(train_sample)) # Print up to 8 examples or fewer if dataset is smaller

    for i in range(num_examples_to_print):
        example = train_sample[i]
        print(f"\n--- Example {i+1} ---")
        for key, value in example.items():
            # Print only the content of each field
            if isinstance(value, str):
                print(f"  {key}: {value}")
            elif isinstance(value, (list, dict)):
                print(f"  {key}: {repr(value)}")
            else:
                print(f"  {key}: {value}")


        if isinstance(supporting_facts, dict):
            print(f"    {repr(supporting_facts)}")
        else:
             print(f"    Type: {type(supporting_facts).__name__} - {repr(supporting_facts)}")


    print("\n" + "=" * 60)
    print("Finished displaying data points.")

else:
    print("train_sample not found. Please run the data processing cell first.")

Displaying 1 data points from train_sample:

--- Example 1 ---
  id: 5ae3cfe05542990afbd1e1e3
  question: Which airport is located in Maine, Sacramento International Airport or Knox County Regional Airport?
  answer: Knox County Regional Airport
  type: comparison
  level: medium
  supporting_facts: {'title': ['Sacramento International Airport', 'Knox County Regional Airport'], 'sent_id': [0, 0]}
  context: {'title': ['Vinalhaven, Maine', 'Owls Head, Maine', 'North Haven, Maine', 'Downeast Flight 46', 'Northern California TRACON', 'Sacramento International Airport', 'Knox County Regional Airport', 'Matinicus Isle, Maine', 'Raleigh Executive Jetport', 'Lea County Regional Airport'], 'sentences': [['Vinalhaven is a town located on the larger of the two Fox Islands in Knox County, Maine, United States.', ' Vinalhaven is also used to refer to the Island itself.', ' The population was 1,165 at the 2010 census.', ' It is home to a thriving lobster fishery and hosts a summer colony.', ' Since

### Create the chain of thought prompt using data points printed from previous cell

In [ ]:
import json
import re # Import re for parsing citations

# Ensure train_sample is available from previous cells
# Create the chain of thought prompt.
# Since we want to control the output of LLM and standardize it, for instance, we want LLM to provide citations in desired format which is a structured output(e.g. json),
#in the prompt we present the instructions, chain of thought instance to be in structured format.

# --- Provided reasoning steps and citations for the first 3 examples ---
# For the three examples we chose, we use Claude 4 to generate the reasoning process and stored the reasoning_steps, citations in lists

prepared_train_sample_indexs = [0,1,2]

prepared_reasoning_steps = [
    [
        "From evidence [23]: Sacramento International Airport is located 10 mi northwest of downtown Sacramento, in Sacramento County, California",
        "From evidence [26]: Knox County Regional Airport is a county owned, public use airport in Knox County, Maine, United States",
        "Since the question asks which airport is in Maine, and Sacramento International Airport is in California while Knox County Regional Airport is in Maine",
        "Therefore, Knox County Regional Airport is the airport located in Maine"
    ],
    [
        "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",
        "From evidence [8]: Russell Hobbs is a manufacturer of household appliances based in Failsworth, Greater Manchester, England",
        "Since Peter Hobbs founded Russell Hobbs, and Russell Hobbs is based in Failsworth",
        "Therefore, the company Peter Hobbs founded is based in Failsworth"
    ],
    [
        "From evidence [22]: Austrolebias bellottii is a species of fish that lives in the basins of the Paraná River and Uruguay River",
        "From evidence [24]: The Uruguay River flows from north to south and forms parts of the boundaries of Brazil, Argentina, and Uruguay",
        "Since Austrolebias bellottii are found in the Uruguay River basin, and the Uruguay River flows from north to south",
        "Therefore, the river flows from north to south"
    ]
]

prepared_citations = [
    "[23], [26]",
    "[7], [8]",
    "[22], [24]"
]

choosen_indices = [1]

provided_train_sample_indexs = [prepared_train_sample_indexs[i] for i in choosen_indices]
provided_reasoning_steps = [prepared_reasoning_steps[i] for i in choosen_indices]
provided_citations = [prepared_citations[1] for i in choosen_indices]

# ------------------------------------------------------------------------

if 'train_sample' not in globals() or len(train_sample) == 0:
    print("❌ train_sample not found or is empty. Please run the data loading and processing cells first.")
else:
    print("Generating Chain-of-Thought prompt instances...")

    cot_exemplars = []



    for i, reasoning_step, provided_citation in zip(provided_train_sample_indexs ,provided_reasoning_steps, provided_citations):
        example = train_sample[i]

        # Print header for the example
        print(f"\n--- Processing Example {i+1} ---")
        print(f"  Question: {example.get('question', '')[:100]}...")

        # Format the context as a list of strings, each including title and sentence
        context_list = []
        linear_index_counter = 1 # Start counter for linear index
        context_sentences_map = {} # Map (title, sent_id) to actual sentence text
        if isinstance(example.get('context'), dict):
            titles = example['context'].get('title', [])
            sentences_lists = example['context'].get('sentences', [])

            # Create a mapping from (title, sent_id) to linear_index for validation
            title_sentence_map = {}
            current_linear_index = 1
            for title_idx, (title, sentences) in enumerate(zip(titles, sentences_lists)):
                 if isinstance(sentences, list):
                      for sent_idx, sentence in enumerate(sentences):
                          context_list.append(f"[{current_linear_index}] Title: {title} - {sentence}")
                          title_sentence_map[(title, sent_idx)] = current_linear_index
                          context_sentences_map[(title, sent_idx)] = sentence # Store sentence text
                          current_linear_index += 1
                 else:
                      context_list.append(f"[{current_linear_index}] Title: {title} - {str(sentences)}")
                      title_sentence_map[(title, 0)] = current_linear_index # Assuming single sentence per title if not list
                      context_sentences_map[(title, 0)] = str(sentences) # Store sentence text
                      current_linear_index += 1

        context_for_pydantic = context_list # Use the list of strings for Contexts


        # Determine reasoning and evidence based on index (using provided for first 3)

          # Use the provided reasoning and parse the provided citations
        reasoning_for_exemplar = reasoning_step

        # Parse provided citations string like "[1], [3]"
        citation_indices = []
        citation_string = provided_citation
        try:
            # Find all numbers within brackets
            found_citations = re.findall(r'\[(\d+)\]', citation_string)
            citation_indices = [int(c) for c in found_citations]

            # Optional: Add validation against ground truth supporting facts linear index
            # This requires mapping ground truth supporting facts to linear indices
            # based on the `title_sentence_map` created earlier.

            # Get ground truth supporting facts from the example
            gold_sf_titles = example.get('supporting_facts', {}).get('title', [])
            gold_sf_sent_ids = example.get('supporting_facts', {}).get('sent_id', [])
            gold_linear_indices = set()

            for sf_title, sf_sent_id in zip(gold_sf_titles, gold_sf_sent_ids):
                if (sf_title, sf_sent_id) in title_sentence_map:
                      gold_linear_indices.add(title_sentence_map[(sf_title, sf_sent_id)])

            # Check if provided citations match gold citations
            provided_indices_set = set(citation_indices)
            if provided_indices_set != gold_linear_indices:
                print(f"⚠️ Warning: Provided citations {provided_indices_set} for example {i+1} do not exactly match ground truth supporting facts {gold_linear_indices}.")
                # Decide whether to use provided or gold. For now, using provided as requested.

        except Exception as e:
            print(f"❌ Error parsing provided citations '{citation_string}' for example {i+1}: {e}. Using empty list.")
            citation_indices = []


        evidence_for_exemplar = citation_indices # Use parsed integer list



        # Create the prompt instance structure
        cot_instance = {
            "instruction": """You are an evidence-grounded QA assistant. Choose the "Supporting Facts" from the "Contexts" given to you and filter out the irrelevant information from the Contexts. Using only the “Supporting Facts,” answer the question. Provide: answer — the short final answer, reasoning — a step-by-step explanation showing how you used the facts, and evidence — a list of citations from the contexts you chose as "Supporting Facts".
    For instance if you choose the first and third sentence as citation from the context, evidence should be [1], [3]. If the facts are insufficient, set answer to “insufficient information”.
     Please ensure that your answer follows this JSON format "output": {
    "answer": "Failsworth",
    "reasoning": [
      "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",
      "From evidence [8]: Russell Hobbs is a manufacturer of household appliances based in Failsworth, Greater Manchester, England",
      "Since Peter Hobbs founded Russell Hobbs, and Russell Hobbs is based in Failsworth",
      "Therefore, the company Peter Hobbs founded is based in Failsworth"
    ],
    "evidence": [
      7,
      8
    ]
  }""",
            "input": {
                "Question": example.get('question', ''),
                "Contexts": context_for_pydantic # Use the list of strings for Contexts
            },
            # Use the determined reasoning and evidence
            "output": {
                "answer": example.get('answer', 'insufficient information'),
                "reasoning": reasoning_for_exemplar,
                "evidence": evidence_for_exemplar # Use the determined list of integers
            }
        }

        # Print the cot_instance first, then print the supporting facts, THEN append
        print(f"\n  --- CoT Instance (JSON) ---")
        print(json.dumps(cot_instance, indent=2))

        print(f"\n  --- Supporting Facts ---")
        supporting_facts = example.get('supporting_facts', {})
        if isinstance(supporting_facts, dict) and 'title' in supporting_facts and 'sent_id' in supporting_facts:
            sf_titles = supporting_facts['title']
            sf_sent_ids = supporting_facts['sent_id']
            for sf_title, sf_sent_id in zip(sf_titles, sf_sent_ids):
                sentence_text = context_sentences_map.get((sf_title, sf_sent_id), "Sentence not found")
                print(f"    Title: '{sf_title}', Sentence ID: {sf_sent_id}, Text: '{sentence_text[:100]}...'")
        else:
             print(f"    Raw Supporting Facts: {repr(supporting_facts)}")

        # Append the cot_instance to the list after printing
        print('cot instance: ', cot_instance)
        cot_exemplars.append(cot_instance)


    # # Print the generated JSON structure (full list)
    # print(f"\n--- Full CoT Exemplars List ---")
    # print(json.dumps(cot_exemplars, indent=2))

    print(f"\n✅ Generated {len(cot_exemplars)} Chain-of-Thought prompt instances.")
    print(f"\n Here is the chain of thought exemplars")
    # Optionally, save this to a file
    output_filename = "chain_of_thought_prompt.json"
    with open(output_filename, 'w') as f:
        json.dump(cot_exemplars, f, indent=2)
    print(f"💾 Saved generated exemplars to '{output_filename}'")

Generating Chain-of-Thought prompt instances...

--- Processing Example 2 ---
  Question: Peter Hobbs founded the company that is based in what town in Manchester?...

  --- CoT Instance (JSON) ---
{
  "instruction": "You are an evidence-grounded QA assistant. Choose the \"Supporting Facts\" from the \"Contexts\" given to you and filter out the irrelevant information from the Contexts. Using only the \u201cSupporting Facts,\u201d answer the question. Provide: answer \u2014 the short final answer, reasoning \u2014 a step-by-step explanation showing how you used the facts, and evidence \u2014 a list of citations from the contexts you chose as \"Supporting Facts\".\n    For instance if you choose the first and third sentence as citation from the context, evidence should be [1], [3]. If the facts are insufficient, set answer to \u201cinsufficient information\u201d.\n     Please ensure that your answer follows this JSON format \"output\": {\n    \"answer\": \"Failsworth\",\n    \"reasoning\

In [ ]:
# Debugging the linear index calculation for evidence
import json

print("🔍 Debugging Linear Index Calculation for Evidence")
print("=" * 60)

# Ensure train_sample is available from previous cells
if 'train_sample' not in globals() or len(train_sample) == 0:
    print("❌ train_sample not found or is empty. Please run the data loading and processing cells first.")
else:
    # Use a few examples for debugging
    num_debug_examples = min(3, len(train_sample))
    debug_examples = train_sample.select(range(num_debug_examples))

    print(f"Testing linear index calculation on {len(debug_examples)} examples:")

    for i, example in enumerate(debug_examples):
        print(f"\n--- Debugging Example {i+1} ---")
        question = example.get('question', '')
        print(f"Question: {question[:100]}...")

        context_data = example.get('context', {})
        supporting_facts_data = example.get('supporting_facts', {})

        if not isinstance(context_data, dict) or not isinstance(supporting_facts_data, dict):
            print("⚠️ Skipping example: Context or Supporting Facts not in expected dict format.")
            continue

        context_titles = context_data.get('title', [])
        context_sentences_lists = context_data.get('sentences', [])
        sf_titles = supporting_facts_data.get('title', [])
        sf_sent_ids = supporting_facts_data.get('sent_id', [])

        # Flatten the context sentences to easily access by linear index
        flat_context_sentences = [sent for sublist in context_sentences_lists for sent in sublist]

        print(f"\nSupporting Facts ({len(sf_titles)} total):")
        for j, (sf_title, sf_sent_id) in enumerate(zip(sf_titles, sf_sent_ids)):
            print(f"  SF {j+1}: Title='{sf_title}', Sentence ID={sf_sent_id}")

            try:
                title_index = context_titles.index(sf_title)

                if title_index < len(context_sentences_lists) and sf_sent_id < len(context_sentences_lists[title_index]):
                    # Calculate the linear index
                    linear_index = sum(len(context_sentences_lists[k]) for k in range(title_index)) + sf_sent_id

                    # Fetch sentence using calculated linear index
                    fetched_sentence = flat_context_sentences[linear_index]

                    # Get the original sentence from supporting facts (for comparison)
                    original_sentence_from_sf = context_sentences_lists[title_index][sf_sent_id]


                    print(f"    Calculated Linear Index (0-based): {linear_index}")
                    print(f"    Fetched Sentence: '{fetched_sentence[:100]}...'")
                    print(f"    Original Sentence from SF: '{original_sentence_from_sf[:100]}...'")

                    # Compare fetched sentence with original sentence from context
                    if fetched_sentence == original_sentence_from_sf:
                        print("    ✅ Verification Successful: Fetched sentence matches original.")
                    else:
                        print("    ❌ Verification Failed: Fetched sentence DOES NOT match original!")
                        print(f"      Fetched: {fetched_sentence}")
                        print(f"      Original: {original_sentence_from_sf}")

                else:
                    print(f"    ⚠️ Skipping SF {j+1}: Sentence ID {sf_sent_id} out of bounds for title '{sf_title}' (has {len(context_sentences_lists[title_index])} sentences).")

            except ValueError:
                print(f"    ⚠️ Skipping SF {j+1}: Title '{sf_title}' not found in context titles.")
            except IndexError:
                 print(f"    ⚠️ Skipping SF {j+1}: Linear index {linear_index} out of bounds for flattened context ({len(flat_context_sentences)} sentences).")
            except Exception as e:
                print(f"    ❌ An unexpected error occurred for SF {j+1}: {e}")


    print(f"\n{'='*60}")
    print("🔍 Debugging complete.")

🔍 Debugging Linear Index Calculation for Evidence
Testing linear index calculation on 3 examples:

--- Debugging Example 1 ---
Question: Which airport is located in Maine, Sacramento International Airport or Knox County Regional Airport?...

Supporting Facts (2 total):
  SF 1: Title='Sacramento International Airport', Sentence ID=0
    Calculated Linear Index (0-based): 22
    Fetched Sentence: 'Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtow...'
    Original Sentence from SF: 'Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtow...'
    ✅ Verification Successful: Fetched sentence matches original.
  SF 2: Title='Knox County Regional Airport', Sentence ID=0
    Calculated Linear Index (0-based): 25
    Fetched Sentence: 'Knox County Regional Airport (IATA: RKD, ICAO: KRKD, FAA LID: RKD) is a county owned, public use air...'
    Original Sentence from SF: 'Knox County Regional Airport (I

## Structural Data validation

In [ ]:
#This code cell provides data validation via Pydantic package.

#The Pydantic package provides schema based data modeling such that,

#we could ensure, structural input and output to the large language model follows a designed schema



from pydantic import BaseModel, Field, ValidationError, validator

from typing import List, Union

import json

import re

import torch



# =============================================

# PYDANTIC DATA MODELS (matching your CoT format)

# =============================================



class QAInput(BaseModel):

    """Input structure matching your CoT format"""

    Question: str

    Contexts: List[str]



class QAOutput(BaseModel):

    """Output structure matching your CoT format"""

    answer: str = Field(description="Short final answer")

    reasoning: Union[str, List[str]] = Field(description="Step-by-step reasoning")

    citations: List[int] = Field(description="Citations like 1, 2")



    @validator('citations', each_item=True)

    def validate_citations(cls, v, values):



        # Get num_contexts from the class-level variable we'll set

        num_contexts = getattr(cls, '_num_contexts', 0)

        if num_contexts <= 0:

            return v



        if v <= 0 or v > num_contexts:

            raise ValueError(f'The citation {v} is not in the contexts. Expected range 1-{num_contexts}')

        return v



    def get_reasoning_steps_count(self) -> int:

        """Count the number of reasoning steps"""

        print(f"🔍 Counting steps in reasoning type: {type(self.reasoning)}")

        print(f"🔍 Reasoning content: {str(self.reasoning)[:100]}...")



        if not self.reasoning:

            return 0



        if isinstance(self.reasoning, str):

            steps = re.findall(r'(?:^\d+\.|^-|^•)', self.reasoning, re.MULTILINE)

            step_count = len(steps) if steps else 1

            print(f"🔍 Found {step_count} steps")

            return step_count

        elif isinstance(self.reasoning, list):

            return len(self.reasoning)

        else:

            return 0



def parse_and_validate_response(raw_response: Union[str, dict], contexts: List[str]) -> QAOutput:

    """Parse and validate response wiwth Pydantic



    Args:

        raw_response: Either a JSON string or a dictionary containing the response

        contexts: List of context strings for validation

    """

    try:

        # Set the number of contexts for validation

        QAOutput._num_contexts = len(contexts)



        # Handle both string and dict inputs

        if isinstance(raw_response, dict):

            parsed = raw_response

        elif isinstance(raw_response, str):

            # Try to extract JSON from response string

            json_match = re.search(r'\{.*\}', raw_response, re.DOTALL)

            if json_match:

                json_str = json_match.group()

                parsed = json.loads(json_str)

            else:

                # Use fallback parsing for non-JSON strings

                print('The answer is not in the format of dict or json, using fallback parse')

                print('The answer is in the format of: ', type(raw_response))

                return fallback_parse(raw_response, contexts)

        else:

            raise ValueError(f"Unsupported input type: {type(raw_response)}")



        print(f'🐛 DEBUG: Parsed keys: {parsed.keys()}')

        return QAOutput(**parsed)



    except (json.JSONDecodeError, ValidationError) as e:

        print(f"❌ Parsing error: {e}")

        print("🔄 Falling back to fallback parser...")

        return fallback_parse(str(raw_response), contexts)


def test_qa_system():

    """Test the QA system with debugging"""



    # Mock response as dictionary (like your actual output)

    mock_response = {

        "answer": "Second Battle of St Albans",

        "reasoning": [

            "I need to find information about Sir Thomas Kyriell's execution and which battle it followed.",

            "From [2], I can see that 'He was executed after the Second Battle of St Albans.'",

            "From [3], I can confirm that 'The Second Battle of St Albans was a battle of the English Wars of the Roses, fought on 17 February 1461.'",

            "This directly answers the question about which battle from the Wars of the Roses preceded his execution."

        ],

        "citations": [2, 3]

    }



    # Mock contexts

    mock_contexts = [

        "[1] Title: Sir Thomas Kyriell - Sir Thomas Kyriell (1396–1461) was an English soldier of the Hundred Years' War and the opening of the Wars of the Roses.",

        "[2] Title: Sir Thomas Kyriell - He was executed after the Second Battle of St Albans.",

        "[3] Title: Second Battle of St Albans - The Second Battle of St Albans was a battle of the English Wars of the Roses, fought on 17 February 1461, at St Albans."

    ]



    try:

        print(f"🐛 DEBUG: Starting test...")

        print(f"📝 Testing input creation...")



        test_input = QAInput(

            Question="Sir Thomas Kyriell was executed after which battle from the Wars of the Roses?",

            Contexts=mock_contexts

        )

        print(f"✅ Input creation successful")



        print(f"🔍 Testing parse_and_validate_response...")

        # Test with dictionary input

        result = parse_and_validate_response(mock_response, mock_contexts)

        print(f"✅ QAOutput created successfully!")



        print("\n📋 RESULTS:")

        print(f"Answer: {result.answer}")

        print(f"Citations: {result.citations}")

        print(f"Reasoning type: {type(result.reasoning)}")

        print(f"Reasoning: {result.reasoning}")



        # Test the method

        if hasattr(result, 'get_reasoning_steps_count'):

            steps_count = result.get_reasoning_steps_count()

            print(f"Number of reasoning steps: {steps_count}")

        else:

            print(f"❌ Method get_reasoning_steps_count not found!")



        print("\n🧪 Testing with JSON string input...")

        # Test with JSON string input

        json_string = json.dumps(mock_response)

        result2 = parse_and_validate_response(json_string, mock_contexts)

        print(f"✅ JSON string parsing successful!")

        print(f"Answer from JSON string: {result2.answer}")

        print(f"Citations from JSON string: {result2.citations}")

        print(f"Reasoning from JSON string: {result2.reasoning}")



        print("\n🧪 Testing with invalid citation...")

        # Test validation with invalid citation

        invalid_response = {

            "answer": "Test",

            "reasoning": ["Test reasoning"],

            "citations": [5]  # Invalid - only 3 contexts available

        }

        try:

            result3 = parse_and_validate_response(invalid_response, mock_contexts)

            print("❌ Should have failed validation!")

        except ValidationError as ve:

            print(f"✅ Validation correctly caught invalid citation: {ve}")



        print("\n✅ All tests completed successfully!")



    except Exception as e:

        print(f"❌ Error: {e}")

        import traceback

        traceback.print_exc()



# Alternative approach using a context manager for cleaner validation

class ValidationContext:

    """Context manager to set validation parameters"""



    def __init__(self, num_contexts: int):

        self.num_contexts = num_contexts



    def __enter__(self):

        QAOutput._num_contexts = self.num_contexts

        return self



    def __exit__(self, exc_type, exc_val, exc_tb):

        if hasattr(QAOutput, '_num_contexts'):

            delattr(QAOutput, '_num_contexts')



def parse_and_validate_with_context(raw_response: Union[str, dict], contexts: List[str]) -> QAOutput:

    """Alternative version using context manager"""

    with ValidationContext(len(contexts)):

        if isinstance(raw_response, dict):

            return QAOutput(**raw_response)

        elif isinstance(raw_response, str):

            json_match = re.search(r'\{.*\}', raw_response, re.DOTALL)

            if json_match:

                parsed = json.loads(json_match.group())

                return QAOutput(**parsed)

            else:

                return fallback_parse(raw_response, contexts)

        else:

            raise ValueError(f"Unsupported input type: {type(raw_response)}")





test_qa_system()

🐛 DEBUG: Starting test...
📝 Testing input creation...
✅ Input creation successful
🔍 Testing parse_and_validate_response...
🐛 DEBUG: Parsed keys: dict_keys(['answer', 'reasoning', 'citations'])
✅ QAOutput created successfully!

📋 RESULTS:
Answer: Second Battle of St Albans
Citations: [2, 3]
Reasoning type: <class 'list'>
Reasoning: ["I need to find information about Sir Thomas Kyriell's execution and which battle it followed.", "From [2], I can see that 'He was executed after the Second Battle of St Albans.'", "From [3], I can confirm that 'The Second Battle of St Albans was a battle of the English Wars of the Roses, fought on 17 February 1461.'", 'This directly answers the question about which battle from the Wars of the Roses preceded his execution.']
🔍 Counting steps in reasoning type: <class 'list'>
🔍 Reasoning content: ["I need to find information about Sir Thomas Kyriell's execution and which battle it followed.", "F...
Number of reasoning steps: 4

🧪 Testing with JSON string inpu

Traceback (most recent call last):
  File "/tmp/ipython-input-3625582184.py", line 173, in parse_and_validate_response
    return QAOutput(**parsed)
           ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pydantic/main.py", line 253, in __init__
    'A custom validator is returning a value other than `self`.\n'
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 1 validation error for QAOutput
citations.0
  Value error, The citation 5 is not in the contexts. Expected range 1-3 [type=value_error, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipython-input-3625582184.py", line 320, in test_qa_system
    result3 = parse_and_validate_response(invalid_response, mock_contexts)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# =============================================
# EXTRACTIVE REASONING HELPER FUNCTIONS
# =============================================

def split_into_sentences(text: str) -> List[str]:
    """
    Split text into sentences using simple regex.

    Args:
        text: Input text to split

    Returns:
        List of sentences
    """
    # Simple sentence splitter - splits on period, exclamation, question mark followed by space
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]


def find_sentence_containing_answer(passage_text: str, answer: str, question: str = "") -> str:
    """
    Find the sentence in passage that best supports the answer.

    Uses keyword overlap scoring to find the most relevant sentence.

    Args:
        passage_text: The full passage text
        answer: The answer string to look for
        question: Optional question for additional context

    Returns:
        The most relevant sentence from the passage
    """
    # Split into sentences
    sentences = split_into_sentences(passage_text)

    if not sentences:
        # Fallback: return first 150 chars if no sentences found
        return passage_text[:150].strip() + ("..." if len(passage_text) > 150 else "")

    # Prepare keywords for scoring
    answer_words = set(answer.lower().split())
    question_words = set(question.lower().split()) if question else set()
    # Remove common stop words from question
    stop_words = {'what', 'where', 'when', 'who', 'which', 'how', 'is', 'are', 'was', 'were',
                  'the', 'a', 'an', 'in', 'on', 'at', 'to', 'for', 'of', 'that', 'this'}
    question_words = question_words - stop_words

    # Score each sentence
    best_sent = sentences[0]  # Default to first sentence
    best_score = 0

    for sent in sentences:
        sent_lower = sent.lower()

        # Count keyword matches (weight answer keywords higher)
        answer_overlap = sum(1 for word in answer_words if word in sent_lower)
        question_overlap = sum(1 for word in question_words if word in sent_lower)

        # Score: answer keywords worth 2 points, question keywords worth 1 point
        score = answer_overlap * 2 + question_overlap

        if score > best_score:
            best_score = score
            best_sent = sent

    return best_sent.strip()


def generate_extractive_reasoning(
    question: str,
    answer: str,
    selected_passages: List[Dict],
    evidence_indices: List[int]
) -> str:
    """
    Generate natural reasoning by extracting relevant sentences from passages.

    This function:
    1. Extracts the most relevant sentence from each evidence passage
    2. Connects them with natural discourse markers
    3. Embeds citations where evidence is used
    4. Adds a conclusion

    Args:
        question: The question being answered
        answer: The correct answer
        selected_passages: List of passage dicts with 'title' and 'text' keys
        evidence_indices: List of 1-indexed passage numbers that support the answer

    Returns:
        Natural reasoning text with embedded citations (30-100 tokens)
    """
    if not evidence_indices or answer == "insufficient context":
        return "Based on the available evidence, I cannot determine a definitive answer to this question."

    # Extract key sentences from each evidence passage
    evidence_sents = []
    for idx in evidence_indices:
        if 1 <= idx <= len(selected_passages):
            passage = selected_passages[idx - 1]  # Convert to 0-indexed
            # Extract most relevant sentence
            key_sent = find_sentence_containing_answer(
                passage['text'],
                answer,
                question
            )
            evidence_sents.append((idx, key_sent))

    if not evidence_sents:
        return f"The answer is {answer}."

    # Build natural reasoning with discourse connectors
    reasoning_parts = []

    # Opening: Frame the task
    reasoning_parts.append("To answer this question,")

    # Middle: Present evidence with natural connectors
    if len(evidence_sents) == 1:
        idx, sent = evidence_sents[0]
        reasoning_parts.append(f"evidence [{idx}] shows that {sent}")

    elif len(evidence_sents) == 2:
        idx1, sent1 = evidence_sents[0]
        idx2, sent2 = evidence_sents[1]
        reasoning_parts.append(f"evidence [{idx1}] shows that {sent1},")
        reasoning_parts.append(f"and evidence [{idx2}] indicates that {sent2}.")

    else:
        # 3+ pieces of evidence
        for i, (idx, sent) in enumerate(evidence_sents):
            if i == 0:
                reasoning_parts.append(f"evidence [{idx}] shows that {sent},")
            elif i < len(evidence_sents) - 1:
                reasoning_parts.append(f"evidence [{idx}] indicates that {sent},")
            else:
                reasoning_parts.append(f"and evidence [{idx}] states that {sent}.")

    # Conclusion: Connect to final answer
    if len(evidence_sents) > 1:
        # Multiple evidence pieces - show synthesis
        citation_list = ", ".join([f"[{idx}]" for idx, _ in evidence_sents])
        reasoning_parts.append(f"Based on {citation_list}, the answer is {answer}.")
    else:
        reasoning_parts.append(f"Therefore, the answer is {answer}.")

    # Join all parts
    reasoning_text = " ".join(reasoning_parts)

    return reasoning_text


print("✅ Extractive reasoning helper functions loaded successfully!")
print("📝 Functions available: split_into_sentences, find_sentence_containing_answer, generate_extractive_reasoning")

✅ Extractive reasoning helper functions loaded successfully!
📝 Functions available: split_into_sentences, find_sentence_containing_answer, generate_extractive_reasoning


## Prompt template building.

In [ ]:
# Data processing functions with curriculum learning
from typing import List, Dict
from pydantic import BaseModel, Field, ValidationError, validator
import json
import re
import torch


# Define instruction and load CoT exemplars for RAG prompting
instruction = """Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its possible that some of the evidences are irrelevant to the question and answer could not find enough sources to support.
 Respond with the answer directly and cite indices like [1], [3]([1] refers to the first evidence provided to you). If the an answer could not be reasoned through the given sources,
say insufficient context.Please give an answer that could only be deduced from the evidences presented to you. If you could not deduce the result from the evidences presented to you, please say insufficient contexts.
Additionally, please keep your output strictly following the JSON format.  "output": {
    "answer": "Failsworth",
    "reasoning": [
      "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",
      "From evidence [8]: Russell Hobbs is a manufacturer of household appliances based in Failsworth, Greater Manchester, England",
      "Since Peter Hobbs founded Russell Hobbs, and Russell Hobbs is based in Failsworth",
      "Therefore, the company Peter Hobbs founded is based in Failsworth"
    ],
    "evidence": [
      7,
      8
    ]
  }
    Please give the direct answer for this case, for answer you dont need to show reasoning, reasoning goes to field "reasoning".
"""

# Load the saved cot exemplar in json format
cot_exemplar_file = "chain_of_thought_prompt.json"
loaded_cot_exemplars = []
try:
    with open(cot_exemplar_file, 'r') as f:
        loaded_cot_exemplars = json.load(f)
    print(f"✅ Successfully loaded {len(loaded_cot_exemplars)} CoT exemplars from '{cot_exemplar_file}'")
    print(f"loaded cot exemplars: ", loaded_cot_exemplars)
    # Demonstrate the structure of a single exemplar
    if loaded_cot_exemplars:
        demonstrate_example = loaded_cot_exemplars[0]
        print("\nStructure of a single exemplar:")
        for key, value in demonstrate_example.items():
            print(f"{key}: {value}")
except FileNotFoundError:
    print(f"❌ Error: CoT exemplar file '{cot_exemplar_file}' not found. Please run the cell to save it first.")
except json.JSONDecodeError as e:
    print(f"❌ Error decoding JSON from '{cot_exemplar_file}': {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred while loading '{cot_exemplar_file}': {e}")









# Function to format a single CoT exemplar into a string for the prompt
def format_cot_exemplar_for_prompt(exemplar_data: Dict) -> str:
    """Formats a single loaded JSON exemplar into a string for the prompt."""
    # This structure should match the desired display within the prompt
    # Example based on the JSON structure:
    input_data = exemplar_data.get("input", {})

    # Use QAInput model for strict validation of the input structure
    try:
        validated_input = QAInput(**input_data)
        # print(f"🐛 DEBUG: Input validated successfully with QAInput.") # Keep debug output minimal
    except ValidationError as e:
        print(f"❌ Input validation failed for exemplar: {e}")
        # Handle validation error - perhaps skip this exemplar or log a warning
        # For now, we'll proceed with the raw data but log the failure
        validated_input = input_data # Use raw data if validation fails


    # Access validated data or raw data if validation failed
    # Format the data into a string for the prompt
    # Ensure contexts is a list before joining
    contexts_list = getattr(validated_input, 'Contexts', input_data.get('Contexts', []))
    if not isinstance(contexts_list, list):
        contexts_list = [] # Ensure it's a list if validation failed or data is malformed

    formatted_input = f"Question: {getattr(validated_input, 'Question', input_data.get('Question', ''))}\nContexts: {'\n'.join(contexts_list)}"


    output_data = exemplar_data.get("output", {})
    # Note: We are NOT validating output here, only formatting it for the prompt string
    formatted_output_reasoning = "\n".join(output_data.get("reasoning", []))
    formatted_output_answer = output_data.get("answer", "insufficient information")
    # Note: We are NOT formatting evidence here for the prompt string as it's part of the output JSON later


    # Construct the example in a way the model can follow, mirroring the intended CoT format
    # The prompt format itself will NOT be a JSON object, but a string that contains structured examples
    return f"""
[Exemplar]
Instruction: {exemplar_data.get("instruction", "").strip()}
Input: {formatted_input.strip()}
Output:
Reasoning:
{formatted_output_reasoning.strip()}
Answer: {formatted_output_answer.strip()}
[/Exemplar]
"""

# Function to create the main prompt template
def create_prompt_template(question: str, passages: List[Dict], building_prompts: Dict, include_answer: bool = True, answer: str = "") -> str:
  """Create standardized prompt template for HotpotQA multihop reasoning
  For now we do not consider batching.
  Adheres to Mistral-7B-Instruct-v0.2 format: <s>[INST] Instruction [/INST] Model response
  Includes optional Chain-of-Thought exemplar after the main instruction.
  """

  # Format evidence section
  evidence_lines = []
  for i, passage in enumerate(passages, 1):
    title = passage.get('title', f'Passage {i}')
    text = passage.get('text', passage.get('passage', ''))
    evidence_lines.append(f"[{i}] Title: {title} - {text}") # Include Title in evidence format
  evidence_text = "\n".join(evidence_lines)

  # Get the formatted CoT exemplars and main instruction
  main_instruction = building_prompts.get('instruction', '').strip()
  cot_exemplar_string = building_prompts.get('cot_exemplar', '').strip()

  # Build the instruction part for the model
  # Include CoT exemplar *after* the main instruction and before Q&A
  instruction_text = f"{cot_exemplar_string}"


  # Build the full prompt with Mistral-Instruct format
  prompt = f"{instruction_text.strip()}<s>[INST]  \n\n{main_instruction}\n\n Now lets keep previous exemplar and instruction in mind but fully focused on solving following question by deducing from the evidences given to you only. [Question]: {question}\n[Evidence]: {evidence_text} [/INST]"

  # Append the expected output format for training
  prompt += "\nOutput:" # Add "Output:" header before the answer part

  if include_answer:
    prompt += f"\n{answer}</s>" # Append the answer for training, on a new line

  return prompt

# Combine loaded exemplars into a single string for the prompt
# This string will be passed as building_prompts['cot_exemplar']
if loaded_cot_exemplars:
    cot_exemplar_string_for_prompt = "\n\n".join([format_cot_exemplar_for_prompt(ex) for ex in loaded_cot_exemplars])
else:
    cot_exemplar_string_for_prompt = ""

# Create the building_prompts dictionary to pass to create_prompt_template
cot_exemplar_string_for_prompt = ''
building_prompts_rag = {'instruction': instruction, 'cot_exemplar': cot_exemplar_string_for_prompt}

print("\n✅ Prompt template building code updated and executed.")



✅ Successfully loaded 1 CoT exemplars from 'chain_of_thought_prompt.json'
loaded cot exemplars:  [{'instruction': 'You are an evidence-grounded QA assistant. Choose the "Supporting Facts" from the "Contexts" given to you and filter out the irrelevant information from the Contexts. Using only the “Supporting Facts,” answer the question. Provide: answer — the short final answer, reasoning — a step-by-step explanation showing how you used the facts, and evidence — a list of citations from the contexts you chose as "Supporting Facts".\n    For instance if you choose the first and third sentence as citation from the context, evidence should be [1], [3]. If the facts are insufficient, set answer to “insufficient information”.\n     Please ensure that your answer follows this JSON format "output": {\n    "answer": "Failsworth",\n    "reasoning": [\n      "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",\n      "From evidence [8]:

In [ ]:
# Calculate the length of the combined instruction and CoT exemplars
if 'building_prompts_rag' in globals():
    instruction_length = len(building_prompts_rag.get('instruction', ''))
    cot_exemplar_length = len(building_prompts_rag.get('cot_exemplar', ''))
    total_prompt_template_length = instruction_length + cot_exemplar_length
    print(f"Length of instruction: {instruction_length}")
    print(f"Length of CoT exemplars string: {cot_exemplar_length}")
    print(f"Total length of prompt template (instruction + exemplars): {total_prompt_template_length}")
else:
    print("building_prompts_rag not found. Please run the relevant cells first.")

Length of instruction: 1363
Length of CoT exemplars string: 0
Total length of prompt template (instruction + exemplars): 1363


##Load tokenizer eval func

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
CACHE_DIR = "/workspace/models" if os.path.exists("/workspace") else "./models"
print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    cache_dir=CACHE_DIR,
    trust_remote_code=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

🔄 Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
for k, v in building_prompts_rag.items():
  print(f"key: {k} \n value: \n{v}")

key: instruction 
 value: 
Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its possible that some of the evidences are irrelevant to the question and answer could not find enough sources to support.
 Respond with the answer directly and cite indices like [1], [3]([1] refers to the first evidence provided to you). If the an answer could not be reasoned through the given sources,
say insufficient context.Please give an answer that could only be deduced from the evidences presented to you. If you could not deduce the result from the evidences presented to you, please say insufficient contexts.
Additionally, please keep your output strictly following the JSON format.  "output": {
    "answer": "Failsworth",
    "reasoning": [
      "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",
      "From evidence [8]: Russell Hobbs is a manufacturer of household appliances based in

## Training data processing

In [ ]:
def process_hotpotqa_for_training(examples, building_prompts: Dict, curriculum_epoch: bool = True, generate_reasoning: bool = False):
    """
    Process HotpotQA examples into training format with structured JSON output.
    Uses extractive reasoning generation with embedded citations.
    """
    processed_examples = []

    i = 0
    # Create a mapping from example ID to its index in the original dataset for exemplar handling
    example_id_to_idx = {ex['id']: idx for idx, ex in enumerate(examples)}

    for example in examples:
        i += 1
        question = example['question']
        answer = example['answer']
        context_data = example['context']
        supporting_facts_data = example['supporting_facts']

        # Create passage list with titles and text
        passages = []
        gold_passages = []

        # STEP 1: Extract gold titles from supporting facts
        gold_facts = set() # Use set of (title, sent_id) tuples
        gold_titles = set()


        try:
            if isinstance(supporting_facts_data, dict):
                # Dict structure: {'title': [...], 'sent_id': [...]}
                if 'title' in supporting_facts_data and 'sent_id' in supporting_facts_data:
                    for title, sent_id in zip(supporting_facts_data['title'], supporting_facts_data['sent_id']):
                        gold_facts.add((title, sent_id))
                        gold_titles.add(title)
            # Removed handling for list structure as systematic investigation shows it's a dict
        except Exception as e:
            # Removed visualization print for this error
            pass


        # STEP 2: Process context to extract passages and map to original (title, sent_id)
        # Also create a map from (title, sent_id) to its sentence text
        context_map = {} # Map (title, sent_id) to sentence text
        passage_list_flat = [] # List of strings: "[idx] Title: ... - Sentence..."
        linear_index_counter = 1
        passage_info_list = [] # List of {title: ..., text: ...}


        try:
            assert isinstance(context_data, dict)
            # HuggingFace dict structure: {'title': [...], 'sentences': [...]}
            if 'title' in context_data and 'sentences' in context_data:
                titles = context_data['title']
                sentences_lists = context_data['sentences']

                for title, sentences in zip(titles, sentences_lists):
                    if isinstance(sentences, list):
                        full_passage_text = " ".join(sentences)
                        passage_info_list.append({"title": title, "text": full_passage_text}) # Store full passage text

                        for sent_idx, sentence in enumerate(sentences):
                            context_map[(title, sent_idx)] = sentence # Map fact to sentence text
                            passage_list_flat.append(f"[{linear_index_counter}] Title: {title} - {sentence}") # Flattened for prompt
                            linear_index_counter += 1
                    else:
                         # Handle cases where sentences is not a list (shouldn't happen based on investigation, but robustness)
                         full_passage_text = str(sentences)
                         passage_info_list.append({"title": title, "text": full_passage_text}) # Store full passage text
                         context_map[(title, 0)] = full_passage_text # Map fact to sentence text
                         passage_list_flat.append(f"[{linear_index_counter}] Title: {title} - {full_passage_text}") # Flattened for prompt
                         linear_index_counter += 1

            # Populate passages and gold_passages lists for selection
            passages = passage_info_list
            gold_passages = [p for p in passages if p['title'] in gold_titles] # Simple check by title for now

        except Exception as e:
            # Removed visualization print for this error
            pass


        # Skip if we couldn't process any passages
        if len(passages) == 0:
            # Removed visualization print for this warning
            continue

        # STEP 3: Curriculum learning strategy
        if curriculum_epoch and len(gold_passages) >= 2:
            # Curriculum: Start with all gold passages + distractors up to 8
            selected_passages = gold_passages.copy()
            distractors = [p for p in passages if p not in gold_passages]
            import random
            random.shuffle(distractors)
            # Ensure we don't exceed 8 total passages
            selected_passages.extend(distractors[:max(0, 8 - len(selected_passages))])
            # Shuffle the selected passages so gold ones aren't always first in the prompt
            random.shuffle(selected_passages)

        else:
            # Standard: Random selection
            import random
            random.shuffle(passages)
            selected_passages = passages[:8]

            # Check if we have enough gold context in the randomly selected passages
            selected_titles = set(p['title'] for p in selected_passages)
            if len(selected_titles.intersection(gold_titles)) < 2 and answer != "insufficient context":
                # If the gold answer exists but insufficient gold context is present in selected passages
                # The target output should reflect insufficient context
                answer = "insufficient context"


        # Ensure selected_passages are present for prompt creation
        if not selected_passages:
             # If somehow no passages were selected, skip this example
             continue

        # STEP 4: Prepare structured output (answer, reasoning, citations) - UPDATED FOR EXTRACTIVE REASONING
        predicted_output = {"reasoning": "", "answer": answer, "citations": []} # Initialize structured output

        if answer != "insufficient context":
            # Find indices of selected passages corresponding to gold facts
            selected_passage_titles = [p['title'] for p in selected_passages]
            selected_passage_texts = [p['text'] for p in selected_passages] # Store full text for matching

            # Build citations list (indices in selected_passages, 1-based)
            citation_indices = set()

            # Map gold facts to indices in the *selected* passages
            for gold_title, gold_sent_id in gold_facts:
                 # Find index of the passage with this gold_title in selected_passages
                 try:
                     # Find all indices where the title matches
                     matching_indices_in_selected = [idx for idx, p in enumerate(selected_passages, 1) if p['title'] == gold_title]

                     if matching_indices_in_selected:
                         # Add matching passage indices to citations
                         citation_indices.update(matching_indices_in_selected)

                 except ValueError:
                     # Gold title not found in selected passages - shouldn't happen if curriculum=True and >=2 gold
                     pass # Or log a warning

            # Ensure unique and sorted citations (indices in selected_passages)
            predicted_output["citations"] = sorted(list(citation_indices))

            # Build reasoning using extractive approach
            original_idx = example_id_to_idx.get(example['id'])

            # Access prepared data for exemplars
            if 'prepared_train_sample_indexs' in globals() and 'prepared_reasoning_steps' in globals():
                try:
                    # Find the position of the current example's original index within the prepared indices
                    exemplar_position = prepared_train_sample_indexs.index(original_idx)
                    # If found, use the pre-defined reasoning (convert list to string if needed)
                    exemplar_reasoning = prepared_reasoning_steps[exemplar_position]
                    if isinstance(exemplar_reasoning, list):
                        # Convert list of reasoning steps to natural paragraph
                        predicted_output["reasoning"] = " ".join(exemplar_reasoning)
                    else:
                        predicted_output["reasoning"] = exemplar_reasoning
                except ValueError:
                    # Not a prepared exemplar, generate extractive reasoning if requested
                    if generate_reasoning and predicted_output["citations"]:
                        # Generate natural extractive reasoning with embedded citations
                        predicted_output["reasoning"] = generate_extractive_reasoning(
                            question=question,
                            answer=answer,
                            selected_passages=selected_passages,
                            evidence_indices=predicted_output["citations"]
                        )
                    elif predicted_output["citations"] and not generate_reasoning:
                        # If citations exist but no reasoning generation requested, add placeholder
                        citation_list = ", ".join([f"[{idx}]" for idx in predicted_output["citations"]])
                        predicted_output["reasoning"] = f"Relevant evidence found in passages {citation_list}."
                    else:
                        # If no citations, reasoning is empty
                        predicted_output["reasoning"] = ""
            else:
                 # If prepared data is not available, generate reasoning or use placeholder
                 if generate_reasoning and predicted_output["citations"]:
                     predicted_output["reasoning"] = generate_extractive_reasoning(
                         question=question,
                         answer=answer,
                         selected_passages=selected_passages,
                         evidence_indices=predicted_output["citations"]
                     )
                 elif predicted_output["citations"]:
                     citation_list = ", ".join([f"[{idx}]" for idx in predicted_output["citations"]])
                     predicted_output["reasoning"] = f"Relevant evidence found in passages {citation_list}."
                 else:
                     predicted_output["reasoning"] = ""


        else:
            # If answer is insufficient context, citations and reasoning should be empty
            predicted_output["citations"] = []
            predicted_output["reasoning"] = "Based on the available evidence, I cannot determine a definitive answer to this question."


        # STEP 5: Create training example with structured JSON output as target
        # Serialize the output dictionary to a JSON string
        try:
            output_json_string = json.dumps(predicted_output, indent=2)
            # Ensure the JSON string follows the desired format for the model output
            # The model should output just the JSON object after [/INST]\nOutput:\n
            target_text = output_json_string

            # The full_text is the prompt + the target_text
            # Corrected variable name from building_prompts to building_prompts_rag
            prompt = create_prompt_template(question, selected_passages, building_prompts, include_answer=False) # Create prompt without the old answer format
            full_text = prompt + "\n" + target_text # Combine prompt and the new JSON target


            if i == 1:
              print('i == 1 DEBUG (Structured Output with Extractive Reasoning)')
              print('question', question)
              # Print only titles and first 100 chars of text for passages
              print('selected_passages (subset):')
              for idx, p in enumerate(selected_passages[:5]): # Print max 5 passages for brevity
                  print(f"  [{idx+1}] {p.get('title', 'N/A')}: {p.get('text', '')[:100]}...")
              if len(selected_passages) > 5:
                   print(f"  ...and {len(selected_passages)-5} more passages")
              print('predicted_output (JSON):\n', json.dumps(predicted_output, indent=2))
              print('input_text (first 400 chars):\n', prompt[:400] + "...")
              print('target_text (JSON string):\n', target_text[:400] + "...")
              print('full_text (first 800 chars):\n', full_text[:800] + "...")


            processed_examples.append({
                "question": question,
                "passages": selected_passages, # Keep passages for potential later use
                "answer": target_text, # Store the JSON string as the 'answer' for consistency with old code expecting 'answer' in eval dataset
                "input_text": prompt,
                "target_text": target_text, # The JSON string is the target
                "full_text": full_text, # Prompt + JSON string
                "has_gold_context": len(gold_passages) >= 2 # Keep track of gold context availability
            })

        except Exception as e:
            print(f"❌ Error creating JSON output for example {i}: {e}")
            # Skip this example if JSON creation fails
            continue


    return Dataset.from_list(processed_examples)

# Process training data with curriculum learning - USING NEW STRUCTURED OUTPUT WITH EXTRACTIVE REASONING
print("📊 Processing HotpotQA data for training with EXTRACTIVE REASONING...")

# Ensure building_prompts_rag is defined before calling this function
# Ensure prepared_train_sample_indexs and prepared_reasoning_steps are available
# They are defined in cell 7agAtJS2Dyxk. Make sure that cell is run first.
if 'building_prompts_rag' in globals() and 'prepared_train_sample_indexs' in globals() and 'prepared_reasoning_steps' in globals():
  # Pass building_prompts_rag and enable reasoning generation for non-exemplars
  train_dataset_curriculum = process_hotpotqa_for_training(train_sample, building_prompts_rag, curriculum_epoch=True, generate_reasoning=True)
  train_dataset_realistic = process_hotpotqa_for_training(train_sample, building_prompts_rag, curriculum_epoch=False, generate_reasoning=True)

  # Evaluation data (realistic setting) - also with structured output as target for evaluation logic
  # The evaluation logic needs to be updated to parse this JSON output
  eval_dataset = process_hotpotqa_for_training(val_sample, building_prompts_rag, curriculum_epoch=False, generate_reasoning=False) # No need to generate reasoning for eval targets

  print(f"✅ Data processed successfully with EXTRACTIVE REASONING:")
  print(f"   Curriculum training: {len(train_dataset_curriculum)} examples")
  print(f"   Realistic training: {len(train_dataset_realistic)} examples")
  print(f"   Evaluation: {len(eval_dataset)} examples")

  # Show sample
  if len(train_dataset_curriculum) > 0:
      sample = train_dataset_curriculum[0]
      print(f"\n📝 Sample training example (with Extractive Reasoning):")
      print(f"Question: {sample['question']}")
      print(f"Answer (JSON string): {sample['answer']}") # This is the JSON string
      print(f"Has gold context: {sample['has_gold_context']}")
      print(f"\n📋 Input text (first 400 chars):")
      print(sample['input_text'][:400] + "...")
      print(f"\n📋 Full text (first 800 chars):")
      print(sample['full_text'][:800] + "...")

  else:
      print("⚠️ No examples processed successfully - investigate data structure further")

  # Log dataset statistics to W&B (only if we have data)
  if len(train_dataset_curriculum) > 0 and 'wandb' in globals() and wandb.run:
      wandb.log({
          "train_curriculum_size": len(train_dataset_curriculum),
          "train_realistic_size": len(train_dataset_realistic),
          "eval_size": len(eval_dataset),
          "gold_context_rate_curriculum": sum(ex['has_gold_context'] for ex in train_dataset_curriculum) / len(train_dataset_curriculum),
          "gold_context_rate_realistic": sum(ex['has_gold_context'] for ex in train_dataset_realistic) / len(train_dataset_realistic)
      })
      print(f"\n✅ All data processed and logged to W&B!")
  elif len(train_dataset_curriculum) > 0:
      print(f"\n⚠️ wandb not initialized. Dataset statistics not logged.")
  else:
      print(f"\n❌ No data processed - check the structure investigation output above")

else:
  print("❌ Required variables (building_prompts_rag, prepared_train_sample_indexs, prepared_reasoning_steps) are not defined. Please run the necessary cells first.")

📊 Processing HotpotQA data for training with EXTRACTIVE REASONING...
i == 1 DEBUG (Structured Output with Extractive Reasoning)
question Which airport is located in Maine, Sacramento International Airport or Knox County Regional Airport?
selected_passages (subset):
  [1] North Haven, Maine: North Haven is a town in Knox County, Maine, United States, in Penobscot Bay.  The town is both a ye...
  [2] Vinalhaven, Maine: Vinalhaven is a town located on the larger of the two Fox Islands in Knox County, Maine, United Stat...
  [3] Lea County Regional Airport: Lea County Regional Airport (IATA: HOB, ICAO: KHOB) (Lea County-Hobbs Airport) is four miles (6.4 km...
  [4] Sacramento International Airport: Sacramento International Airport (IATA: SMF, ICAO: KSMF, FAA LID: SMF) is 10 mi northwest of downtow...
  [5] Downeast Flight 46: Downeast Airlines Flight 46 was a scheduled airline service in the United States from Boston's Logan...
  ...and 3 more passages
predicted_output (JSON):
 {
  "reason

In [ ]:
#compute the percentage of insufficient context in the training dataset
print(type(train_dataset_curriculum))
train_dataset = train_dataset_curriculum.to_list()
print(len(train_dataset))
num_samples = len(train_dataset)
num_insufficient_context = sum(1 for ex in train_dataset if ex['has_gold_context'])

<class 'datasets.arrow_dataset.Dataset'>
2000


## Eval Function, Wandb training integration

### Eval Function

In [ ]:
# Comprehensive HotpotQA Evaluator with Robust Tensor Handling and Utility Functions

import torch

import torch.nn as nn

import numpy as np

import pandas as pd

import json

import os

import zipfile

import shutil

from pathlib import Path

import time

import gc

from typing import Dict, List, Optional, Tuple

import warnings

warnings.filterwarnings('ignore')



# Core ML libraries (should work on cloud platforms)

from transformers import (

    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,

    TrainingArguments, Trainer, TrainerCallback, TrainerState

)

from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

from datasets import Dataset, load_dataset

import evaluate

import wandb



# Define necessary variables if not already defined (for standalone execution)

if 'MAX_SEQ_LENGTH' not in globals():

    MAX_SEQ_LENGTH = 1000 # Default value


class HotpotQAEvaluator:

    """Comprehensive evaluator for HotpotQA multihop reasoning"""



    def __init__(self):

        pass



    def normalize_answer(self, text):

        """Normalize answer text for comparison"""

        import re

        import string



        # Convert to lowercase

        text = text.lower()



        # Remove articles

        text = re.sub(r'\b(a|an|the)\b', ' ', text)



        # Remove punctuation

        text = text.translate(str.maketrans('', '', string.punctuation))



        # Remove extra whitespace

        text = ' '.join(text.split())



        return text



    def answer_f1_score(self, prediction, ground_truth):

        """Calculate F1 score between prediction and ground truth"""

        from collections import Counter



        pred_tokens = self.normalize_answer(prediction).split()

        gold_tokens = self.normalize_answer(ground_truth).split()



        if len(pred_tokens) == 0 and len(gold_tokens) == 0:

            return 1.0

        if len(pred_tokens) == 0 or len(gold_tokens) == 0:

            return 0.0



        common_tokens = Counter(pred_tokens) & Counter(gold_tokens)

        num_same = sum(common_tokens.values())



        if num_same == 0:

            return 0.0



        precision = num_same / len(pred_tokens)

        recall = num_same / len(gold_tokens)



        return 2 * precision * recall / (precision + recall)



    def answer_exact_match(self, prediction, ground_truth):

        """Calculate exact match score"""

        return float(self.normalize_answer(prediction) == self.normalize_answer(ground_truth))



# Initialize evaluator

evaluator = HotpotQAEvaluator()




def convert_predictions_to_token_ids(predictions):

    """Robust conversion of any prediction format to token IDs with detailed debugging"""



    print(f"\n🔍 TENSOR CONVERSION DEBUG:")

    print(f"   Input type: {type(predictions)}")

    print(f"   Input class: {predictions.__class__.__name__}")



    if hasattr(predictions, 'shape'):

        print(f"   Shape: {predictions.shape}")

    elif hasattr(predictions, '__len__'):

        print(f"   Length: {len(predictions)}")



    if hasattr(predictions, 'dtype'):

        print(f"   Dtype: {predictions.dtype}")



    # Sample first few values for inspection

    if isinstance(predictions, (list, tuple)):

        print(f"   First element type: {type(predictions[0])}")

        if hasattr(predictions[0], 'shape'):

            print(f"   First element shape: {predictions[0].shape}")

        elif hasattr(predictions[0], '__len__'):

            print(f"   First element length: {len(predictions[0])}")



        # Show actual values (first few)

        if hasattr(predictions[0], '__iter__') and not isinstance(predictions[0], str):

            try:

                sample_vals = list(predictions[0])[:3] if len(predictions[0]) > 0 else []

                print(f"   Sample values from first element: {sample_vals}")

            except:

                print(f"   Could not extract sample values")



    elif hasattr(predictions, 'flatten'):

        try:

            flat_sample = predictions.flatten()[:3].tolist()

            print(f"   Sample flattened values: {flat_sample}")

        except:

            print(f"   Could not flatten for sampling")



    # Now attempt conversion

    print(f"   🔧 Attempting conversion...")



    # Case 1: Already token IDs (integers)

    if hasattr(predictions, 'dtype') and predictions.dtype in [torch.int32, torch.int64, torch.long]:

        print(f"   ✅ Already token IDs (integers)")

        return predictions



    # Case 2: Logits (floats) - need argmax

    if hasattr(predictions, 'dtype') and predictions.dtype in [torch.float16, torch.float32, torch.bfloat16]:

        print(f"   🎯 Converting logits (floats) using argmax")

        if len(predictions.shape) == 3:  # [batch, seq_len, vocab_size]

            print(f"   📊 3D tensor [batch, seq_len, vocab_size] -> argmax on dim=-1")

            result = torch.argmax(predictions, dim=-1)

            print(f"   ✅ Converted to shape: {result.shape}")

            return result

        elif len(predictions.shape) == 2:  # Already [batch, seq_len]

            print(f"   📊 2D tensor [batch, seq_len] -> converting to long")

            result = predictions.long()

            print(f"   ✅ Converted to dtype: {result.dtype}")

            return result

        else:

            print(f"   ⚠️ Unexpected tensor shape: {predictions.shape}")

            result = predictions.long()

            return result



    # Case 3: Numpy arrays

    if isinstance(predictions, np.ndarray):

        print(f"   🔢 Converting numpy array")

        if predictions.dtype in [np.float16, np.float32, np.float64]:

            print(f"   🎯 Numpy float array")

            if len(predictions.shape) == 3:

                print(f"   📊 3D numpy array -> argmax on axis=-1")

                result = torch.tensor(np.argmax(predictions, axis=-1))

                print(f"   ✅ Converted to torch tensor shape: {result.shape}")

                return result

            else:

                print(f"   📊 Converting numpy float to torch long")

                result = torch.tensor(predictions).long()

                return result

        else:

            print(f"   📊 Converting numpy int to torch long")

            result = torch.tensor(predictions).long()

            return result



    # Case 4: Nested lists

    if isinstance(predictions, list):

        print(f"   📝 Processing list input")

        if len(predictions) > 0:

            if isinstance(predictions[0], list):

                print(f"   📊 Nested list structure")

                try:

                    tensor = torch.tensor(predictions)

                    print(f"   🔄 Converted to tensor: {tensor.shape}, dtype: {tensor.dtype}")

                    if tensor.dtype in [torch.float16, torch.float32]:

                        if len(tensor.shape) == 3:

                            print(f"   🎯 3D float tensor -> argmax")

                            return torch.argmax(tensor, dim=-1)

                        else:

                            print(f"   🔄 Converting float tensor to long")

                            return tensor.long()

                    else:

                        print(f"   ✅ Already integer tensor")

                        return tensor.long()

                except Exception as e:

                    print(f"   ⚠️ Tensor conversion failed: {e}")

                    # Fallback: flatten

                    print(f"   🔄 Attempting flatten fallback")

                    flat = [item for sublist in predictions for item in sublist]

                    result = torch.tensor(flat).long()

                    print(f"   ✅ Flattened result shape: {result.shape}")

                    return result

            else:

                print(f"   📊 Simple list -> tensor")

                result = torch.tensor(predictions).long()

                print(f"   ✅ Converted shape: {result.shape}")

                return result



    # Fallback: try to convert directly

    print(f"   🆘 Using fallback conversion")

    try:

        result = torch.tensor(predictions).long()

        print(f"   ✅ Fallback successful: {result.shape}")

        return result

    except Exception as e:

        print(f"   ❌ Fallback failed: {e}")

        raise e



### This function is used in inference demo or debugging model performance with quick question answering
def generate_answer(question: str, passages: List[Dict], building_prompt:Dict, model_to_use, max_new_tokens: int = 1000) -> str:

    """Generate answer using specified model"""



    # Create prompt

    prompt = create_prompt_template(question, passages, building_prompt ,include_answer=False)

    print(f"Prompt length: {len(prompt)}")

    # Tokenize

    inputs = tokenizer(

        prompt,

        return_tensors="pt",

        truncation=True,

        max_length=MAX_SEQ_LENGTH - max_new_tokens

    ).to(model_to_use.device)

    print('the maximum sequence length is: ',MAX_SEQ_LENGTH)

    # Generate

    with torch.no_grad():

        outputs = model_to_use.generate(

            **inputs,

            max_new_tokens=max_new_tokens,

            temperature=0.1,

            do_sample=True,

            pad_token_id=tokenizer.eos_token_id,

            eos_token_id=tokenizer.eos_token_id

        )



    # Decode response (only new tokens)

    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)



    # Validate structure output of response using the dedicated function

    try:

        # Create a list of context strings in the format expected by parse_and_validate_response

        context_strings = [f"[{i+1}] Title: {p.get('title', '')} - {p.get('text', '')}" for i, p in enumerate(passages)]

        print('the response type is: ', type(response))

        validated_response = parse_and_validate_response(response, context_strings)

        # If you need the raw string response for later steps, return that.

        # If you need the validated object, return validated_response.

        # For now, returning the original string response as the rest of the code expects it.

    except Exception as e:

        print(f"❌ Response validation failed: {e}")

        # Handle validation failure - maybe return an error message or the raw response

        # For now, just print the error and continue, returning the raw response.

        pass





    #print the length of the generated answer

    print(f"Generated answer length: {len(response)}") # Use raw response length for consistency

    return response.strip()







# Data collator for instruction tuning

class HotpotQADataCollator:

    """Custom data collator for HotpotQA instruction tuning"""



    def __init__(self, tokenizer, max_length: int = 2048):

        self.tokenizer = tokenizer

        self.max_length = max_length

        self.visualization_count = 0 # Add visualization counter

        self.max_visualization_prints = 3 # Limit prints



    def __call__(self, examples: List[Dict]) -> Dict[str, torch.Tensor]:

        # Extract full text (input + target)

        texts = [ex['full_text'] for ex in examples]



        # Tokenize

        batch = self.tokenizer(

            texts,

            truncation=True,

            padding=True,

            max_length=self.max_length,

            return_tensors="pt"

        )



        # Create labels (same as input_ids, but with -100 for padding)

        labels = batch["input_ids"].clone()



        # Mask padding tokens in labels

        labels[labels == self.tokenizer.pad_token_id] = -100



        # For instruction tuning, mask the input part and only train on answer

        for i, example in enumerate(examples):

            input_text = example['input_text']

            # Tokenize input_text separately to get its length in tokens

            input_ids_input_text = self.tokenizer(input_text, add_special_tokens=False)["input_ids"]

            input_length = len(input_ids_input_text)



            # Mask input tokens in labels (only train on answer)

            if input_length < len(labels[i]):

                labels[i][:input_length] = -100



            # Visualization prints for the first few examples in the batch

            if self.visualization_count < self.max_visualization_prints:

                print(f"\n--- Example {self.visualization_count+1} (HotpotQADataCollator) ---")

                print(f"  Full Text (first 400 chars): {example['full_text'][:400]}...")

                print(f"  Input Text Length (tokens): {input_length}")

                print(f"  Tokenized Input IDs (first 20): {batch['input_ids'][i][:20].tolist()}")

                print(f"  Labels Before Masking Input (first 20): {batch['input_ids'][i][:20].tolist()}") # Same as input_ids

                print(f"  Labels After Masking Input (first 20): {labels[i][:20].tolist()}")

                # Find first non -100 label to show where target starts

                first_target_token_idx = (labels[i] != -100).nonzero(as_tuple=True)[0][0] if (labels[i] != -100).any() else -1

                print(f"  First Target Token Index in Labels: {first_target_token_idx}")

                # Show a snippet around the masking boundary

                snippet_start = max(0, input_length - 5)

                snippet_end = min(len(labels[i]), input_length + 5)

                print(f"  Labels around input_length {input_length} (indices {snippet_start}-{snippet_end-1}): {labels[i][snippet_start:snippet_end].tolist()}")



                self.visualization_count += 1





        batch["labels"] = labels

        return batch



# Create data collator

data_collator = HotpotQADataCollator(tokenizer, max_length=MAX_SEQ_LENGTH)



print("✅ Comprehensive evaluation with ROBUST TENSOR HANDLING and UTILITY FUNCTIONS ready!")

print("📊 Features:")

print("   - Handles all tensor formats (logits, token IDs, numpy, lists)")

print("   - Detailed debugging output for tensor analysis")

print("   - Graceful error handling with full context")

print("   - HotpotQA-specific metrics (F1, EM, Citation Accuracy)")



✅ Comprehensive evaluation with ROBUST TENSOR HANDLING and UTILITY FUNCTIONS ready!
📊 Features:
   - Handles all tensor formats (logits, token IDs, numpy, lists)
   - Detailed debugging output for tensor analysis
   - Graceful error handling with full context
   - HotpotQA-specific metrics (F1, EM, Citation Accuracy)
   - Includes generate_answer and evaluate_model_on_dataset for flexible evaluation


In [ ]:
# Unified Evaluation Function for Comprehensive Model Assessment
from tqdm import tqdm
import torch
from typing import Dict, List, Optional, Any
import sys

def evaluate_model_comprehensive(
    model,
    tokenizer,
    eval_dataset,
    evaluator,
    model_name: str = "Model",
    max_examples: Optional[int] = None,
    use_rag_prompting: bool = True,
    verbose_level: str = "summary",  # "all", "sample", "summary"
    wandb_prefix: Optional[str] = None,
    building_prompts: Optional[Dict] = None
) -> Dict[str, Any]:
    """
    Unified evaluation function for both baseline and fine-tuned models.

    Added example_idx to extract_answer_and_citations calls
    Added separator prints before extraction for better debugging
    Fixed "insufficient context" EM/F1 scoring
    Display now shows extracted answer, not raw response

    Args:
        model: Model to evaluate (base or fine-tuned)
        tokenizer: Tokenizer
        eval_dataset: Dataset to evaluate on
        evaluator: HotpotQAEvaluator instance
        model_name: Name for logging
        max_examples: Max examples to evaluate (None = all)
        use_rag_prompting: If True, use RAG prompts; if False, use direct JSON format
        verbose_level: "all" (print every example), "sample" (first 5), "summary" (final only)
        wandb_prefix: Prefix for W&B metrics (e.g., "baseline_rag" or "final_eval")
        building_prompts: Prompt template dict (required if use_rag_prompting=True)

    Returns:
        Dictionary with comprehensive metrics
    """

    model.eval()
    device = next(model.parameters()).device

    # Select dataset subset if specified
    if max_examples:
        eval_subset = eval_dataset.select(range(min(max_examples, len(eval_dataset))))
    else:
        eval_subset = eval_dataset

    # Metrics tracking
    f1_scores = []
    em_scores = []
    citation_precisions = []
    citation_recalls = []
    citation_f1s = []

    # Insufficient context tracking
    insufficient_context_count = 0
    insufficient_context_correct = 0
    per_example_results = []

    print(f"\n{'='*80}")
    print(f"🔍 Evaluating {model_name} on {len(eval_subset)} examples...")
    print(f"{'='*80}\n")

    for idx, example in enumerate(tqdm(eval_subset, desc=f"Evaluating {model_name}")):
        try:
            question = example.get('question', '')
            passages = example.get('passages', [])

            # Create input based on prompting strategy
            if use_rag_prompting:
                if building_prompts is None:
                    raise ValueError("building_prompts required when use_rag_prompting=True")
                input_text = create_prompt_template(question, passages, building_prompts, include_answer=False)
            else:
                # Use direct input_text from dataset (for fine-tuned model)
                input_text = example.get('input_text', '')

            # Generate prediction
            inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=2048)
            inputs = {k: v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=300,
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

            # FIXED: Print separator BEFORE extraction with flush
            if verbose_level == "all" or (verbose_level == "sample" and idx < 5):
                print(f"\n{'='*60}", flush=True)
                print(f"--- Example {idx + 1} ---", flush=True)
                print(f"Question: {question[:100]}...", flush=True)
                sys.stdout.flush()

            # FIXED: Extract answer and citations from response with example_idx
            pred_answer, pred_citations = extract_answer_and_citations(response, example_idx=idx)

            # Parse ground truth
            gt_text = example.get('answer', '{}')
            gold_answer, gold_citations = extract_answer_and_citations(gt_text, example_idx=idx)

            # Compute metrics
            f1 = evaluator.answer_f1_score(pred_answer, gold_answer)
            em = evaluator.answer_exact_match(pred_answer, gold_answer)

            # FIXED: Special handling for "insufficient context" cases
            # When both answers are "insufficient context", the F1 and EM should be 1.0
            is_insufficient = gold_answer.lower().strip() == 'insufficient context'
            pred_insufficient = pred_answer.lower().strip() == 'insufficient context'

            if is_insufficient and pred_insufficient:
                # Both correctly identified as insufficient context
                # This is a PERFECT match, so override the scores
                f1 = 1.0
                em = 1.0

            # Citation metrics
            if gold_citations:
                pred_set = set(pred_citations)
                gold_set = set(gold_citations)

                if pred_set:
                    citation_precision = len(pred_set & gold_set) / len(pred_set)
                else:
                    citation_precision = 0.0

                citation_recall = len(pred_set & gold_set) / len(gold_set)

                if citation_precision + citation_recall > 0:
                    citation_f1 = 2 * citation_precision * citation_recall / (citation_precision + citation_recall)
                else:
                    citation_f1 = 0.0
            else:
                # No gold citations (insufficient context case)
                # If model also predicts no citations, that's perfect (1.0)
                # If model predicts citations when there shouldn't be any, that's wrong (0.0)
                citation_precision = 1.0 if not pred_citations else 0.0
                citation_recall = 1.0
                citation_f1 = 1.0 if not pred_citations else 0.0

            # Insufficient context tracking
            if is_insufficient:
                insufficient_context_count += 1
                if pred_insufficient:
                    insufficient_context_correct += 1

            # Store results
            f1_scores.append(f1)
            em_scores.append(em)
            citation_precisions.append(citation_precision)
            citation_recalls.append(citation_recall)
            citation_f1s.append(citation_f1)

            per_example_results.append({
                'question': question,
                'predicted_answer': pred_answer,
                'gold_answer': gold_answer,
                'predicted_citations': pred_citations,
                'gold_citations': gold_citations,
                'f1': f1,
                'em': em,
                'citation_precision': citation_precision,
                'citation_recall': citation_recall,
                'citation_f1': citation_f1
            })

            # FIXED: Verbose output with extracted answers (not raw response)
            if verbose_level == "all" or (verbose_level == "sample" and idx < 5):
                print(f"Predicted: {pred_answer}", flush=True)
                print(f"Gold: {gold_answer}", flush=True)
                print(f"Pred Citations: {pred_citations}", flush=True)
                print(f"Gold Citations: {gold_citations}", flush=True)
                print(f"F1: {f1:.3f}, EM: {em:.3f}, Citation F1: {citation_f1:.3f}", flush=True)
                sys.stdout.flush()

        except Exception as e:
            print(f"\n⚠️  Error on example {idx}: {str(e)}", flush=True)
            continue

    # Compute final metrics
    results = {
        'em': np.mean(em_scores) if em_scores else 0.0,
        'f1': np.mean(f1_scores) if f1_scores else 0.0,
        'citation_precision': np.mean(citation_precisions) if citation_precisions else 0.0,
        'citation_recall': np.mean(citation_recalls) if citation_recalls else 0.0,
        'citation_f1': np.mean(citation_f1s) if citation_f1s else 0.0,
        'insufficient_context_rate': insufficient_context_correct / insufficient_context_count if insufficient_context_count > 0 else 0.0,
        'insufficient_context_total': insufficient_context_count,
        'insufficient_context_correct': insufficient_context_correct,
        'total_examples': len(per_example_results),
        'per_example_results': per_example_results
    }

    # Print summary
    print(f"\n{'='*80}")
    print(f"📊 {model_name.upper()} - EVALUATION RESULTS")
    print(f"{'='*80}")
    print(f"Total Examples: {results['total_examples']}")
    print(f"Exact Match (EM): {results['em']:.3f}")
    print(f"F1 Score: {results['f1']:.3f}")
    print(f"Citation Precision: {results['citation_precision']:.3f}")
    print(f"Citation Recall: {results['citation_recall']:.3f}")
    print(f"Citation F1: {results['citation_f1']:.3f}")
    print(f"Insufficient Context Detection: {results['insufficient_context_rate']:.1%} ({results['insufficient_context_correct']}/{results['insufficient_context_total']})")
    print(f"{'='*80}\n")

    # Log to W&B
    if wandb_prefix and wandb.run:
        wandb.log({
            f"{wandb_prefix}_em": results['em'],
            f"{wandb_prefix}_f1": results['f1'],
            f"{wandb_prefix}_citation_precision": results['citation_precision'],
            f"{wandb_prefix}_citation_recall": results['citation_recall'],
            f"{wandb_prefix}_citation_f1": results['citation_f1'],
            f"{wandb_prefix}_insufficient_context_rate": results['insufficient_context_rate'],
        })

    return results

print("✅ Unified evaluation function loaded successfully!")
print("🔧 FIXES APPLIED:")
print("   • Added example_idx parameter to extract_answer_and_citations")
print("   • Added flush=True to all debug prints for proper tqdm alignment")
print("   • Fixed 'insufficient context' scoring: EM=1.0, F1=1.0 when both match")
print("   • Display now shows extracted answer instead of raw response")


✅ Unified evaluation function loaded successfully!
🔧 FIXES APPLIED:
   • Added example_idx parameter to extract_answer_and_citations
   • Added flush=True to all debug prints for proper tqdm alignment
   • Fixed 'insufficient context' scoring: EM=1.0, F1=1.0 when both match
   • Display now shows extracted answer instead of raw response


In [ ]:
# 🔧 CRITICAL EVALUATION FIXES - Run this cell to fix all evaluation bugs!
# This cell overrides the buggy functions in Cell 34 and Cell 22
# FIXED: Now handles both "citations" and "evidence" fields
# FIXED: Added flush=True to fix output buffering issues
# FIXED: Added example_idx parameter for better debugging

import json
import re
import sys
from typing import Tuple, List, Union, Dict

def extract_answer_and_citations(generated_text: str, example_idx: int = None) -> Tuple[str, List[int]]:
    """
    Extract answer and citations from generated text, prioritizing JSON parsing.

    Handles malformed JSON, extra text after JSON, and attempts text fallback.
    Also handles answer potentially being a list in the model output.
    FIXED: Now checks both "citations" and "evidence" fields.
    FIXED: Added example_idx for better debugging and flush=True for output alignment.

    Args:
        generated_text: Model output string
        example_idx: Optional example index for debugging output

    Returns:
        Tuple of (answer: str, citations: List[int])
    """
    # Reduce verbosity - only show first 200 chars
    debug_text = generated_text[:200].replace('\n', ' ')
    prefix = f"[Ex {example_idx}] " if example_idx is not None else ""
    print(f"{prefix}🔍 Extracting from: {debug_text}...", flush=True)

    try:
        # Method 1: Try strict JSON parse first (best case)
        parsed = json.loads(generated_text)
        answer = parsed.get('answer', '')
        # Handle answer being a list or other type
        if isinstance(answer, list):
             answer = ", ".join(str(a) for a in answer) # Join list elements into a string
        elif not isinstance(answer, str):
             answer = str(answer) # Convert other types to string
        answer = answer.strip() # Apply strip after ensuring it's a string

        # FIXED: Check both 'citations' and 'evidence' fields
        citations = parsed.get('citations', parsed.get('evidence', []))
        # Ensure citations are integers and unique
        citations = sorted(list(set(int(c) for c in citations if isinstance(c, (int, str)) and str(c).isdigit())))

        print(f"{prefix}✅ JSON parse OK: answer='{answer[:50]}...', citations={citations}", flush=True)
        return answer, citations

    except (json.JSONDecodeError, ValueError, TypeError) as e:
        print(f"{prefix}⚠️  JSON parse failed: {str(e)[:50]}. Trying fallback...", flush=True)

        # Method 2: Try to find the JSON-like part and parse it
        json_match = re.search(r'\{.*\}', generated_text, re.DOTALL)
        if json_match:
            json_substring = json_match.group(0) # Get the matched substring
            print(f"{prefix}🔍 Found JSON substring, parsing...", flush=True)
            try:
                parsed_substring = json.loads(json_substring)
                answer = parsed_substring.get('answer', '')
                # Handle answer being a list or other type in substring
                if isinstance(answer, list):
                    answer = ", ".join(str(a) for a in answer)
                elif not isinstance(answer, str):
                    answer = str(answer)
                answer = answer.strip() # Apply strip after ensuring it's a string

                # FIXED: Check both 'citations' and 'evidence' fields
                citations = parsed_substring.get('citations', parsed_substring.get('evidence', []))
                citations = sorted(list(set(int(c) for c in citations if isinstance(c, (int, str)) and str(c).isdigit())))
                print(f"{prefix}✅ Substring parse OK: answer='{answer[:50]}...', citations={citations}", flush=True)
                return answer, citations
            except (json.JSONDecodeError, ValueError, TypeError) as sub_e:
                 print(f"{prefix}⚠️  Substring parse failed. Using regex...", flush=True)

        # Method 3: Fallback to regex on raw text (less robust but handles some cases)
        print(f"{prefix}🔄 Using regex fallback...", flush=True)

        # Attempt to find answer using regex
        answer_match = re.search(r'"answer"\s*:\s*("([^"]*)"|\[.*?\])', generated_text, re.DOTALL) # Added capture for list
        if answer_match:
            # Check if it's a string or a list match
            if answer_match.group(2): # String match
                 answer = answer_match.group(2) # Don't strip yet
            elif answer_match.group(1): # List match (capture group 1 contains the list string)
                 list_str = answer_match.group(1) # Don't strip yet
                 try:
                     # Attempt to parse the list string
                     answer_list = json.loads(list_str)
                     if isinstance(answer_list, list):
                          answer = ", ".join(str(a) for a in answer_list)
                     else:
                          answer = str(answer_list) # Should ideally be a list
                 except:
                      answer = list_str # If list parsing fails, just use the string representation
            else:
                 answer = generated_text[:100] # Default if regex finds something unexpected, don't strip yet
        else:
            # Fallback if no "answer": "..." or "answer": [...] found
            # Try to extract the first reasonable looking text chunk
            answer = generated_text.split('\n')[0] # Take the first line, don't strip yet
            if len(answer) > 100: answer = answer[:100] + "..."

        answer = answer.strip() # Apply strip after all possibilities

        # FIXED: Look for both "citations" and "evidence" arrays using regex
        citations_match = re.search(r'"citations"\s*:\s*\[([\d,\s]+)\]', generated_text)
        if not citations_match:
            # Try "evidence" if "citations" not found
            citations_match = re.search(r'"evidence"\s*:\s*\[([\d,\s]+)\]', generated_text)

        citations = []
        if citations_match:
            citations_str = citations_match.group(1)
            citations = sorted(list(set(int(c.strip()) for c in citations_str.split(',') if c.strip().isdigit())))

        print(f"{prefix}📝 Regex result: answer='{answer[:50]}...', citations={citations}", flush=True)

        return answer, citations


def fallback_parse(raw_response: str, contexts: List[str]) -> QAOutput:
    """
    Fallback parser for malformed responses within the QAOutput structure.
    This is called by parse_and_validate_response when the full JSON parse fails.

    FIXED: Attempts JSON parse first, then uses regex fallback.
    Ensures reasoning is a string.
    Now handles both "citations" and "evidence" fields.

    Args:
        raw_response: Raw model output string
        contexts: List of context passages (for validation)

    Returns:
        QAOutput object with answer, reasoning (str), and citations
    """
    print("🔄 fallback_parse called...", flush=True)
    debug_text = raw_response[:150].replace('\n', ' ')
    print(f"   Input: {debug_text}...", flush=True)

    # Set num_contexts for validation BEFORE attempting to create QAOutput
    # This needs to be done on the QAOutput class itself
    if hasattr(QAOutput, '_num_contexts'):
        original_num_contexts = QAOutput._num_contexts
    else:
        original_num_contexts = 0
    QAOutput._num_contexts = len(contexts)

    try:
        # Attempt JSON parse first
        parsed = json.loads(raw_response)
        print("✅ fallback_parse: JSON OK", flush=True)

        # Normalize reasoning to string if it's a list or other type
        reasoning = parsed.get('reasoning', '')
        if isinstance(reasoning, list):
            reasoning = ' '.join(str(step) for step in reasoning)
        elif not isinstance(reasoning, str):
             reasoning = str(reasoning)

        # FIXED: Check both 'citations' and 'evidence' fields
        citations = parsed.get('citations', parsed.get('evidence', []))
        citations = [int(c) for c in citations if isinstance(c, (int, str)) and str(c).isdigit()]

        # Create QAOutput - Validation will happen here
        qa_output = QAOutput(
            answer=parsed.get('answer', 'insufficient context'),
            reasoning=reasoning,
            citations=citations
        )
        print("✅ QAOutput created from JSON", flush=True)
        return qa_output

    except (json.JSONDecodeError, ValidationError, ValueError, TypeError) as e:
        print(f"⚠️ fallback_parse: JSON failed, using regex", flush=True)

        # Regex fallback on the raw string
        answer, citations = extract_answer_and_citations(raw_response) # Re-use the regex logic from extract_answer_and_citations
        reasoning = "" # In this deep fallback, we don't try to reconstruct reasoning from unstructured text

        # Ensure citations are within bounds using the set _num_contexts
        valid_citations = [c for c in citations if 1 <= c <= len(contexts)]
        if len(valid_citations) != len(citations):
             print(f"⚠️ Filtered {len(citations) - len(valid_citations)} invalid citations", flush=True)

        # Create QAOutput - Validation will happen again, but with filtered citations
        try:
            qa_output = QAOutput(
                answer=answer,
                reasoning=reasoning,
                citations=valid_citations # Use filtered citations
            )
            print("✅ QAOutput created from regex", flush=True)
            return qa_output
        except ValidationError as final_e:
            print(f"❌ Final validation failed: {final_e}", flush=True)
            # Return a minimal QAOutput indicating failure
            return QAOutput(answer="parsing error", reasoning="", citations=[])

    finally:
        # Restore original _num_contexts or clean up
        if hasattr(QAOutput, '_num_contexts'):
             if original_num_contexts > 0:
                  QAOutput._num_contexts = original_num_contexts
             else:
                  delattr(QAOutput, '_num_contexts')


# Test the fix
print("🔧 Testing the REVISED extract_answer_and_citations()...", flush=True)

# Add test for "evidence" field
test_response_with_evidence = """{
  "answer": "Kurt Weill",
  "reasoning": ["Step 1", "Step 2"],
  "evidence": [7, 1]
}"""

print("\n--- Testing 'evidence' field (the actual issue!) ---", flush=True)
answer, citations = extract_answer_and_citations(test_response_with_evidence)
print(f"   Answer: '{answer}'", flush=True)
print(f"   Citations: {citations}", flush=True)
assert answer == "Kurt Weill" and citations == [1, 7], f"FAILED: got '{answer}' and {citations}"
print("✅ PASS", flush=True)

# Quick tests for key scenarios
test_response_good = """{"answer": "Gimme Shelter", "citations": [1, 7]}"""
test_response_answer_list = """{"answer": ["Item 1", "Item 2"], "citations": [6]}"""

print("\n--- Testing 'citations' field ---", flush=True)
answer, citations = extract_answer_and_citations(test_response_good)
assert answer == "Gimme Shelter" and citations == [1, 7]
print("✅ PASS", flush=True)

print("\n--- Testing answer as list ---", flush=True)
answer, citations = extract_answer_and_citations(test_response_answer_list)
assert answer == "Item 1, Item 2" and citations == [6]
print("✅ PASS", flush=True)

print("\n✅ REVISED parsing functions now handle both 'citations' and 'evidence' fields!", flush=True)
print("📝 Reduced debug verbosity for cleaner evaluation output", flush=True)
print("🔧 Added flush=True to fix output buffering with tqdm progress bar", flush=True)


🔧 Testing the REVISED extract_answer_and_citations()...

--- Testing 'evidence' field (the actual issue!) ---
🔍 Extracting from: {   "answer": "Kurt Weill",   "reasoning": ["Step 1", "Step 2"],   "evidence": [7, 1] }...
✅ JSON parse OK: answer='Kurt Weill...', citations=[1, 7]
   Answer: 'Kurt Weill'
   Citations: [1, 7]
✅ PASS

--- Testing 'citations' field ---
🔍 Extracting from: {"answer": "Gimme Shelter", "citations": [1, 7]}...
✅ JSON parse OK: answer='Gimme Shelter...', citations=[1, 7]
✅ PASS

--- Testing answer as list ---
🔍 Extracting from: {"answer": ["Item 1", "Item 2"], "citations": [6]}...
✅ JSON parse OK: answer='Item 1, Item 2...', citations=[6]
✅ PASS

✅ REVISED parsing functions now handle both 'citations' and 'evidence' fields!
📝 Reduced debug verbosity for cleaner evaluation output
🔧 Added flush=True to fix output buffering with tqdm progress bar


### wandb integration for saving finetuned adapter

In [ ]:
# W&B Checkpoint Management (Artifact-based, <500MB)
def save_adapter_only(peft_model, output_dir: str, max_shard_size: str = "400MB") -> str:
    """Save only LoRA adapter weights, compress to zip"""
    os.makedirs(output_dir, exist_ok=True)

    # Save adapter weights only
    peft_model.save_pretrained(
        output_dir,
        max_shard_size=max_shard_size,
        safe_serialization=True
    )

    # Create zip file
    zip_path = f"{output_dir}.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, arcname)

    # Get zip size
    zip_size_mb = os.path.getsize(zip_path) / 1024 / 1024
    print(f"📦 Adapter zip created: {zip_path} ({zip_size_mb:.1f} MB)")

    if zip_size_mb > 500:
        print(f"⚠️ Warning: Zip size {zip_size_mb:.1f} MB exceeds 500MB limit")

    return zip_path

def upload_adapter_artifact(
    wandb_run,
    zip_path: str,
    aliases: List[str],
    metadata: Dict
) -> str:
    """Upload adapter zip as W&B artifact"""

    artifact = wandb.Artifact(
        name="qlora-adapters",
        type="model",
        description="QLoRA adapter weights for Mistral-7B HotpotQA fine-tuning",
        metadata=metadata
    )

    # Add the zip file
    artifact.add_file(zip_path)

    # Log artifact with aliases
    wandb_run.log_artifact(artifact, aliases=aliases)

    print(f"📤 Uploaded artifact with aliases: {aliases}")
    return artifact.id

def download_and_restore_adapter(wandb_run, artifact_alias: str = "latest") -> Optional[str]:
    """Download adapter from W&B artifact and restore"""
    try:
        # Get artifact
        artifact = wandb_run.use_artifact(f"qlora-adapters:{artifact_alias}")
        artifact_dir = artifact.download()

        # Find zip file
        zip_files = [f for f in os.listdir(artifact_dir) if f.endswith('.zip')]
        if not zip_files:
            print(f"❌ No zip file found in artifact {artifact_alias}")
            return None

        zip_path = os.path.join(artifact_dir, zip_files[0])

        # Extract zip
        extract_dir = zip_path.replace('.zip', '_extracted')
        with zipfile.ZipFile(zip_path, 'r') as zipf:
            zipf.extractall(extract_dir)

        print(f"📥 Downloaded and extracted adapter from {artifact_alias}")
        return extract_dir

    except Exception as e:
        print(f"❌ Failed to download artifact {artifact_alias}: {e}")
        return None

class WandBCheckpointCallback(TrainerCallback):
    """Custom callback for W&B artifact management"""

    def __init__(self, wandb_run, output_dir: str = "./checkpoints"):
        self.wandb_run = wandb_run
        self.output_dir = output_dir
        self.best_metric = 0.0

    def on_save(self, args, state, control, model=None, **kwargs):
        """Called when checkpoint is saved"""
        print('saving checkpoint to wandb on_save')
        if model is None:
            return

        # Create checkpoint directory
        checkpoint_dir = os.path.join(self.output_dir, f"checkpoint-{state.global_step}")

        try:
            # Save adapter and create zip
            zip_path = save_adapter_only(model, checkpoint_dir)

            # Upload with 'latest' alias
            metadata = {
                "step": state.global_step,
                "epoch": state.epoch,
                "learning_rate": state.log_history[-1].get("learning_rate", 0) if state.log_history else 0,
                "train_loss": state.log_history[-1].get("train_loss", 0) if state.log_history else 0,
                "base_model": "mistralai/Mistral-7B-Instruct-v0.2"
            }

            upload_adapter_artifact(
                self.wandb_run,
                zip_path,
                aliases=["latest"],
                metadata=metadata
            )

            # Cleanup local files to save space
            shutil.rmtree(checkpoint_dir, ignore_errors=True)
            os.remove(zip_path)

        except Exception as e:
            print(f"❌ Failed to save/upload checkpoint: {e}")

    def on_evaluate(self, args, state, control, model=None, logs=None, **kwargs):
        """Called after evaluation"""
        if model is None or logs is None:
            return

        # Check if this is the best model so far
        current_metric = logs.get("eval_f1", 0.0)

        if current_metric > self.best_metric:
            self.best_metric = current_metric
            print(f"🏆 New best model! F1: {current_metric:.4f}")

            # Save and upload as 'best'
            checkpoint_dir = os.path.join(self.output_dir, f"best-checkpoint-{state.global_step}")

            try:
                zip_path = save_adapter_only(model, checkpoint_dir)

                metadata = {
                    "step": state.global_step,
                    "epoch": state.epoch,
                    "eval_f1": current_metric,
                    "eval_em": logs.get("eval_em", 0.0),
                    "eval_citation_acc": logs.get("eval_citation_acc", 0.0),
                    "base_model": "mistralai/Mistral-7B-Instruct-v0.2"
                }

                upload_adapter_artifact(
                    self.wandb_run,
                    zip_path,
                    aliases=["best", "latest"],
                    metadata=metadata
                )

                # Cleanup
                shutil.rmtree(checkpoint_dir, ignore_errors=True)
                os.remove(zip_path)

            except Exception as e:
                print(f"❌ Failed to save/upload best checkpoint: {e}")

print("💾 W&B Checkpoint management ready!")
print("📋 Features:")
print("   - Adapter-only saves (never full base model)")
print("   - Compressed artifacts <500MB")
print("   - Aliases: 'latest' and 'best'")
print("   - Resume capability from artifacts")

💾 W&B Checkpoint management ready!
📋 Features:
   - Adapter-only saves (never full base model)
   - Compressed artifacts <500MB
   - Aliases: 'latest' and 'best'
   - Resume capability from artifacts


# Prompt Generation Approach

##  Baseline Evaluation: RAG Prompting (Pre-training)

This section evaluates the base Mistral-7B-Instruct model using RAG prompting strategy before fine-tuning.

### Loading Mistral-7B-instruct

In [ ]:
# Release memory from previously loaded model if it exists
import gc
import torch

print("🧹 Attempting to clear GPU memory...")

# --- Check memory BEFORE cleanup ---
if torch.cuda.is_available():
    allocated_before = torch.cuda.memory_allocated() / 1024**3
    cached_before = torch.cuda.memory_reserved() / 1024**3
    print(f"   GPU Memory BEFORE cleanup:")
    print(f"     Allocated: {allocated_before:.2f} GB")
    print(f"     Cached: {cached_before:.2f} GB")
else:
    print("   CUDA not available, skipping memory checks.")


if 'model' in globals() and model is not None:
    try:
        print("   Deleting 'model' variable...")
        del model
        print("   Deleted 'model' variable.")
    except Exception as e:
        print(f"   Error deleting model: {e}")

# Force garbage collection
gc.collect()

# Clear CUDA cache
if torch.cuda.is_available():
    print("   Attempting to clear CUDA cache...")
    torch.cuda.empty_cache()
    print("   Cleared CUDA cache.")
else:
    print("   CUDA not available, skipping cache clear.")
print("✅ Memory clear attempt complete.")

# --- Check memory AFTER cleanup (before loading new model) ---
if torch.cuda.is_available():
    allocated_after_cleanup = torch.cuda.memory_allocated() / 1024**3
    cached_after_cleanup = torch.cuda.memory_reserved() / 1024**3
    print(f"   GPU Memory AFTER cleanup (before loading new model):")
    print(f"     Allocated: {allocated_after_cleanup:.2f} GB")
    print(f"     Cached: {cached_after_cleanup:.2f} GB")
    print(f"   Memory reduction (Allocated): {allocated_before - allocated_after_cleanup:.2f} GB")
    print(f"   Memory reduction (Cached): {cached_before - cached_after_cleanup:.2f} GB")


# Model configuration - Mistral-7B-Instruct-v0.2 with persistent cache
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
LORA_RANK = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1


# Cache directory for RunPod persistence (will be preserved across sessions)
CACHE_DIR = "/workspace/models" if os.path.exists("/workspace") else "./models"

print(f"🔧 Loading model: {MODEL_NAME}")
print(f"📐 LoRA Config: rank={LORA_RANK}, alpha={LORA_ALPHA}, dropout={LORA_DROPOUT}")
print(f"💾 Cache directory: {CACHE_DIR}")

# Create cache directory if it doesn't exist
os.makedirs(CACHE_DIR, exist_ok=True)

# Check if we're authenticated with HuggingFace (required for Mistral)
try:
    from huggingface_hub import whoami
    user_info = whoami()
    print(f"✅ HuggingFace authenticated as: {user_info['name']}")
except Exception as e:
    print(f"⚠️ HuggingFace authentication required for Mistral model")
    print(f"   Run: huggingface-cli login")
    print(f"   Or set HF_TOKEN environment variable")
    print(f"   Error: {e}")

# 8-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

if "tokenizer" not in globals():
  print("🔄 Loading tokenizer...")
  tokenizer = AutoTokenizer.from_pretrained(
      MODEL_NAME,
      cache_dir=CACHE_DIR,
      trust_remote_code=True
  )
  if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
else:
  print('tokenizer is already here')

print("🔄 Loading quantized model with use_cache=False...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16, # Keep bfloat16 for compute
    cache_dir=CACHE_DIR,
    trust_remote_code=True,
    use_cache=False # Explicitly set use_cache to False
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA configuration for Mistral architecture
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # attention modules
        "gate_proj", "up_proj", "down_proj",     # MLP modules
    ],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Add LoRA adapters
print("🔄 Adding LoRA adapters...")
model = get_peft_model(model, lora_config)

# Print model info
model.print_trainable_parameters()

# Calculate model size
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Model Statistics:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Trainable %: {100 * trainable_params / total_params:.2f}%")
print(f"   Memory footprint: ~{total_params * 1 / 1024**3:.1f} GB (8-bit)") # Estimate for 8-bit


print("✅ Mistral-7B model loaded with persistent cache!")
print(f"💾 Model cached at: {CACHE_DIR}")
print("🔄 Ready for QLoRA training on RTX A5000")

# --- Check memory AFTER loading new model ---
if torch.cuda.is_available():
    allocated_after_load = torch.cuda.memory_allocated() / 1024**3
    cached_after_load = torch.cuda.memory_reserved() / 1024**3
    print(f"\n   GPU Memory AFTER loading new model:")
    print(f"     Allocated: {allocated_after_load:.2f} GB")
    print(f"     Cached: {cached_after_load:.2f} GB")

🧹 Attempting to clear GPU memory...
   GPU Memory BEFORE cleanup:
     Allocated: 0.00 GB
     Cached: 0.00 GB
   Attempting to clear CUDA cache...
   Cleared CUDA cache.
✅ Memory clear attempt complete.
   GPU Memory AFTER cleanup (before loading new model):
     Allocated: 0.00 GB
     Cached: 0.00 GB
   Memory reduction (Allocated): 0.00 GB
   Memory reduction (Cached): 0.00 GB
🔧 Loading model: mistralai/Mistral-7B-Instruct-v0.2
📐 LoRA Config: rank=16, alpha=32, dropout=0.1
💾 Cache directory: ./models
✅ HuggingFace authenticated as: jeffgong11235
tokenizer is already here
🔄 Loading quantized model with use_cache=False...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

🔄 Adding LoRA adapters...
trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758

📊 Model Statistics:
   Total parameters: 7,283,675,136
   Trainable parameters: 41,943,040
   Trainable %: 0.58%
   Memory footprint: ~6.8 GB (8-bit)
✅ Mistral-7B model loaded with persistent cache!
💾 Model cached at: ./models
🔄 Ready for QLoRA training on RTX A5000

   GPU Memory AFTER loading new model:
     Allocated: 7.64 GB
     Cached: 8.42 GB


In [ ]:
# Calculate the length of the combined instruction and CoT exemplars
if 'building_prompts_rag' in globals():
    instruction_length = len(building_prompts_rag.get('instruction', ''))
    cot_exemplar_length = len(building_prompts_rag.get('cot_exemplar', ''))
    total_prompt_template_length = instruction_length + cot_exemplar_length
    print(f"Length of instruction: {instruction_length}")
    print(f"Length of CoT exemplars string: {cot_exemplar_length}")
    print(f"Total length of prompt template (instruction + exemplars): {total_prompt_template_length} characters.")
else:
    print("Variable 'building_prompts_rag' is not defined in the current environment. Please run the relevant cells first.")

Length of instruction: 1363
Length of CoT exemplars string: 0
Total length of prompt template (instruction + exemplars): 1363 characters.


In [ ]:
print(f"cot_exemplar: {building_prompts_rag.get('cot_exemplar', '')}")
print(f"instruction: {building_prompts_rag.get('instruction', '')}")

cot_exemplar: 
instruction: Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its possible that some of the evidences are irrelevant to the question and answer could not find enough sources to support.
 Respond with the answer directly and cite indices like [1], [3]([1] refers to the first evidence provided to you). If the an answer could not be reasoned through the given sources,
say insufficient context.Please give an answer that could only be deduced from the evidences presented to you. If you could not deduce the result from the evidences presented to you, please say insufficient contexts.
Additionally, please keep your output strictly following the JSON format.  "output": {
    "answer": "Failsworth",
    "reasoning": [
      "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell",
      "From evidence [8]: Russell Hobbs is a manufacturer of household appliances based i

## Demo testing

In [ ]:
# Debugging Low Scores: Display Examples
print("🔍 Debugging Low Scores: Inspecting Model Outputs")
print("=" * 70)


# print the model used for inference
print("model config: ", model.config)


# Select a few examples from the evaluation dataset
num_debug_examples = 5  # You can adjust this number
debug_examples = eval_dataset.select(range(min(num_debug_examples, len(eval_dataset))))

print(f"📝 Displaying {len(debug_examples)} examples from the evaluation set:")

for i, example in enumerate(debug_examples):
    print(f"\n" + "="*80)
    print(f"📝 EXAMPLE {i+1}")
    print(f"="*80)

    print(f"❓ Question: {example['question']}")
    print(f"✅ Gold Answer: {example['answer']}")

    print(f"\n📚 Provided Passages:")
    for j, passage in enumerate(example['passages'], 1):
        print(f"   [{j}] {passage['title']}: {passage['text']} have ")

    # Get the fine-tuned model's prediction for this example
    chain_of_thought_prediction = generate_answer(example['question'], example['passages'], building_prompts_rag, model)

    print(f"\n🤖 Non finetuned Model Prediction:")
    print(f"   {chain_of_thought_prediction}")
    print(f"Non finetuned Model Prediction finished")

    # You can manually compare the "Gold Answer" and "Fine-tuned Model Prediction"
    # to understand discrepancies and potential issues.

print(f"\n" + "="*80)
print(f"🔍 Debugging examples displayed. Analyze the outputs above to identify patterns in errors.")
#   [2] Oliver Reed: Robert Oliver Reed (13 February 1938 – 2 May 1999) was an English actor known for his upper-middle class, macho image, hellraiser lifestyle,
#and "tough guy" roles.  Notable films include "The Trap" (1966), "Oliver! " (1968), "Women in Love" (1969), "Hannibal Brooks" (1969), "The Devils" (1971),
#"The Three Musketeers" (1973), "Tommy" (1975), "Lion of the Desert" (1981), "Castaway" (1986), "The Adventures of Baron Munchausen" (1988) and "Funny Bones" (1995).
# For "Gladiator" (2000), his final film, Reed was posthumously nominated for the BAFTA Award for Best Actor in a Supporting Role. have


#We need to ensure the validation process is correct




🔍 Debugging Low Scores: Inspecting Model Outputs
model config:  MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "bfloat16",
  "eos_token_id": 2,
  "head_dim": null,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": false,
    "_load_in_8bit": true,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-0

Turn this into markdown: As we could see that prompting the model even with clear instructions and exemplars for in-context learning, the model still struggles to follow the pattern to answer the question. For instance, we want direct answer without explanation, but the model struggles on this. Morever, it seems the model finds it difficult to know when the context given to it is in-sufficient for answering the question. The citation are also none complete. Before considering domain-specific instruction tuning or supervised finetuning, lets try to fully evaluate the prompt approach given its simplicity.



This example demonstrate the difficulty of controlling model output style:


Question: Who released the song "With or Without You" first, Jai McDowall or U2?
 Gold Answer: U2 [5, 8]
Model Prediction:
   Answer: U2 released the song "With or Without You" first.
Reasoning:
From evidence [5], U2 released the song "With or Without You" as the lead single from their fifth studio album "The Joshua Tree" in 1987.
From evidence [8], Jai McDowall released a promotional single of the same name, "With or Without You," from his debut album "Believe" in 2011.
Therefore, U2's release of the song predates Jai McDowall's by over 14 years.
Evidence: [5], [8]

Comments: As you can see the answer of ground truth is U2 but yours is not direct, i want direct anwer like U2.


### Baseline Performance Metrics

Comprehensive evaluation of baseline model on evaluation dataset.

In [ ]:
# Pre-Training Baseline Evaluation with RAG Prompting
# Evaluate base Mistral-7B-Instruct model before fine-tuning

print("🔍 Starting baseline evaluation with RAG prompting approach...")
print(f"   Model: Mistral-7B-Instruct (base, no fine-tuning)")
print(f"   Strategy: RAG with few-shot exemplars")
print(f"   Dataset: First 100 examples from eval_dataset\n")
baseline_model = model
# Evaluate using unified function
baseline_results = evaluate_model_comprehensive(
    model=baseline_model,
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
    evaluator=evaluator,
    model_name="Baseline RAG Prompting",
    max_examples=200,  # Evaluate on first 100 examples
    use_rag_prompting=True,
    verbose_level="sample",  # Print first 5 examples
    wandb_prefix="baseline_rag",
    building_prompts=building_prompts_rag
)

# Store for later comparison
print("✅ Baseline evaluation complete!")
print(f"📊 Key Results:")
print(f"   • Exact Match: {baseline_results['em']:.1%}")
print(f"   • F1 Score: {baseline_results['f1']:.3f}")
print(f"   • Citation F1: {baseline_results['citation_f1']:.3f}")
print(f"   • Insufficient Context Detection: {baseline_results['insufficient_context_rate']:.1%}")


🔍 Starting baseline evaluation with RAG prompting approach...
   Model: Mistral-7B-Instruct (base, no fine-tuning)
   Strategy: RAG with few-shot exemplars
   Dataset: First 100 examples from eval_dataset


🔍 Evaluating Baseline RAG Prompting on 200 examples...



Evaluating Baseline RAG Prompting:   0%|          | 0/200 [00:00<?, ?it/s]


--- Example 1 ---
Question: What nationality was Oliver Reed's character in the film Royal Flash?...
[Ex 0] 🔍 Extracting from: {   "answer": "Otto von Bismarck (German)",   "reasoning": [     "From evidence [1]: Oliver Reed acted as Otto von Bismarck in the film 'Royal Flash'",     "From historical evidence [5]: Otto von Bism...
[Ex 0] ✅ JSON parse OK: answer='Otto von Bismarck (German)...', citations=[1, 5]
[Ex 0] 🔍 Extracting from: {   "reasoning": "From evidence [23]: Sacramento International Airport is located 10 mi northwest of downtown Sacramento, in Sacramento County, California From evidence [26]: Knox County Regional Airp...
[Ex 0] ✅ JSON parse OK: answer='Prussian...', citations=[1, 5]
Predicted: Otto von Bismarck (German)
Gold: Prussian
Pred Citations: [1, 5]
Gold Citations: [1, 5]
F1: 0.000, EM: 0.000, Citation F1: 1.000


Evaluating Baseline RAG Prompting:   0%|          | 1/200 [00:25<1:23:12, 25.09s/it]


--- Example 2 ---
Question: Pacific Mozart Ensemble performed which German composer's Der Lindberghflug in 2002?...
[Ex 1] 🔍 Extracting from: 18th century English novel by Henry Mackenzie, "The Man of Feeling".  The play was written for the Berliner Ensemble and premiered on 3 April 1950.  The play's themes include the nature of morality an...
[Ex 1] ⚠️  JSON parse failed: Extra data: line 1 column 3 (char 2). Trying fallback...
[Ex 1] 🔄 Using regex fallback...
[Ex 1] 📝 Regex result: answer='18th century English novel by Henry Mackenzie, "Th...', citations=[]
[Ex 1] 🔍 Extracting from: {   "reasoning": "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell From evidence [8]: Russell Hobbs is a manufacturer of household applian...
[Ex 1] ✅ JSON parse OK: answer='Kurt Julian Weill...', citations=[1, 8]
Predicted: 18th century English novel by Henry Mackenzie, "The Man of Feeling".  The play was written for the B...
Gold: Kurt Julia

...

[Ex 198] 🔍 Extracting from: {     "answer": "Brian Billick",     "reasoning": [       "From evidence [1] and [8]: Brian Billick was the head coach of the Baltimore Ravens from 1999 to 2007 and specifically from the 6th season of...
[Ex 198] ✅ JSON parse OK: answer='Brian Billick...', citations=[1, 8]
[Ex 198] 🔍 Extracting from: {   "reasoning": "Relevant evidence found in passages [1], [8].",   "answer": "6th season",   "citations": [     1,     8   ] }...
[Ex 198] ✅ JSON parse OK: answer='6th season...', citations=[1, 8]


Evaluating Baseline RAG Prompting: 100%|█████████▉| 199/200 [1:57:55<00:36, 36.10s/it]

[Ex 199] 🔍 Extracting from: {     "answer": "Vocelli Pizza and Noble Roman's serve pizza",     "evidence": [2, 5]   }  Reasoning: [   "From evidence [2]: Vocelli Pizza is a pizzeria",   "From evidence [5]: Noble Roman's is a piz...
[Ex 199] ⚠️  JSON parse failed: Extra data: line 6 column 1 (char 91). Trying fallback...
[Ex 199] 🔍 Found JSON substring, parsing...
[Ex 199] ✅ Substring parse OK: answer='Vocelli Pizza and Noble Roman's serve pizza...', citations=[2, 5]
[Ex 199] 🔍 Extracting from: {   "reasoning": "Relevant evidence found in passages [2], [5].",   "answer": "Pizza",   "citations": [     2,     5   ] }...
[Ex 199] ✅ JSON parse OK: answer='Pizza...', citations=[2, 5]


Evaluating Baseline RAG Prompting: 100%|██████████| 200/200 [1:58:24<00:00, 35.52s/it]


📊 BASELINE RAG PROMPTING - EVALUATION RESULTS
Total Examples: 200
Exact Match (EM): 0.175
F1 Score: 0.274
Citation Precision: 0.458
Citation Recall: 0.703
Citation F1: 0.402
Insufficient Context Detection: 11.7% (9/77)

✅ Baseline evaluation complete!
📊 Key Results:
   • Exact Match: 17.5%
   • F1 Score: 0.274
   • Citation F1: 0.402
   • Insufficient Context Detection: 11.7%


# Model Finetuning

In [ ]:
# Training Configuration - Fixed for compatibility and memory optimization
LEARNING_RATE = 1e-4
NUM_EPOCHS = 2
SAVE_STEPS = 200
LOGGING_STEPS = 50
WARMUP_STEPS = 100
OUTPUT_DIR = "./qlora-checkpoints"

# Calculate realistic training time
effective_batch_size = BATCH_SIZE * GRAD_ACCUM_STEPS
steps_per_epoch = TRAIN_SIZE // effective_batch_size
total_steps = steps_per_epoch * NUM_EPOCHS
estimated_hours = total_steps * 0.1 / 60  # Rough estimate: 0.1 min per step

print(f"🎯 Training Configuration (Memory Optimized):")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE} (effective: {effective_batch_size})")
print(f"   Max Seq Length: {MAX_SEQ_LENGTH}")
print(f"   Save Steps: {SAVE_STEPS}")
print(f"   Steps per epoch: {steps_per_epoch}")
print(f"   Total steps: {total_steps}")
print(f"   💰 Estimated time: ~{estimated_hours:.1f} hours")
print(f"   🚫 Early stopping: DISABLED (fixes memory issues)")

# Training arguments - EVALUATION DISABLED to prevent memory issues
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_steps=WARMUP_STEPS,
    max_grad_norm=1.0,
    weight_decay=0.01,

    # Logging - EVALUATION DISABLED
    logging_steps=LOGGING_STEPS,
    eval_strategy="no",  # DISABLED: Prevents CUDA OOM during training
    save_steps=SAVE_STEPS,
    save_strategy="steps",

    # Model selection - DISABLED since no evaluation during training
    save_total_limit=2,  # Keep last 2 checkpoints
    # load_best_model_at_end=False,  # Disabled (no evaluation to determine "best")
    # metric_for_best_model=None,    # Disabled
    # greater_is_better=None,        # Disabled

    # Precision - trying fp16 for better compatibility
    fp16=True,  # More compatible than bf16
    dataloader_pin_memory=False,

    # W&B integration
    report_to="wandb",
    run_name=RUN_NAME,

    # Other optimizations
    remove_unused_columns=False,
    dataloader_num_workers=2,
)

# Create callback - adjusted for no early stopping
wandb_callback = WandBCheckpointCallback(run, OUTPUT_DIR)

# Initialize trainer - no compute_metrics needed since eval is disabled
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_curriculum,
    eval_dataset=eval_dataset,  # Still needed for post-training evaluation
    data_collator=data_collator,
    callbacks=[wandb_callback],
)

print(f"\n✅ Training arguments configured (evaluation disabled)!")
print(f"📊 Estimated training time: ~{estimated_hours:.1f} hours")
print(f"💰 Estimated cost: ${estimated_hours * HOURLY_RATE:.2f}")
print(f"🎯 Fixed schedule: {NUM_EPOCHS} epochs with curriculum learning")
print(f"💾 Memory optimized: No evaluation during training")
print(f"✅ Trainer initialized successfully!")

# Memory check before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    allocated = torch.cuda.memory_allocated() / 1024**3
    cached = torch.cuda.memory_reserved() / 1024**3
    print(f"\n💾 GPU Memory before training:")
    print(f"   Allocated: {allocated:.2f} GB")
    print(f"   Cached: {cached:.2f} GB")
    # print(f"   Available: {vram_gb - cached:.2f} GB")

In [ ]:
# Training Loop with Curriculum Learning
print("🏋️ Starting QLoRA training with curriculum learning...")
print(f"🎯 Target: Improve Answer F1 score on HotpotQA multihop reasoning")
print(f"⏱️ Estimated time: {len(train_dataset_curriculum) * NUM_EPOCHS / (BATCH_SIZE * GRAD_ACCUM_STEPS) / 100:.1f}+ hours")
print(f"\n{'='*60}")
print(f"🚀 TRAINING STARTED - Monitor at: {run.url}")
print(f"{'='*60}")

# Record start time
start_time = time.time()

try:
    # Phase 1: Curriculum learning with forced gold passages
    print(f"\n📚 PHASE 1: Curriculum Learning (forced gold passages)")
    print(f"   Gold context rate: {sum(ex['has_gold_context'] for ex in train_dataset_curriculum) / len(train_dataset_curriculum):.2%}")

    trainer.train_dataset = train_dataset_curriculum

    # Start training for 1 epoch
    initial_epochs = 1
    training_args.num_train_epochs = initial_epochs
    trainer.args = training_args
    trainer.train()

    # --- Manually save checkpoint after Phase 1 ---
    print("\n💾 Saving checkpoint after Phase 1...")
    trainer.save_model(os.path.join(OUTPUT_DIR, f"checkpoint-phase1-end-step-{trainer.state.global_step}"))
    # Trigger W&B artifact upload for this specific checkpoint
    if wandb_callback:
        wandb_callback.on_save(trainer.args, trainer.state, trainer.control, model=trainer.model)
    print("✅ Checkpoint saved after Phase 1.")
    # -----------------------------------------------

    print(f"\n🎯 PHASE 2: Realistic Training (gold may be missing)")
    print(f"   Gold context rate: {sum(ex['has_gold_context'] for ex in train_dataset_realistic) / len(train_dataset_realistic):.2%}")

    # Switch to realistic dataset for final epoch
    trainer.train_dataset = train_dataset_realistic

    # Continue training for remaining epochs
    # Note: Setting num_train_epochs here means total epochs will be initial_epochs + remaining epochs if starting from scratch
    # If resuming, trainer automatically handles epoch counting.
    # For manual phase control, let's train for the difference
    remaining_epochs = NUM_EPOCHS - initial_epochs
    if remaining_epochs > 0:
        print(f"Continuing for {remaining_epochs} more epochs...")
        for epoch in range(initial_epochs, NUM_EPOCHS):
            print(f"\n🏋️ Starting Phase 2, Epoch {epoch + 1}/{NUM_EPOCHS}")
            trainer.train() # This continues training until trainer.state.epoch reaches NUM_EPOCHS


    # Training completed successfully
    end_time = time.time()
    training_time = end_time - start_time

    print(f"\n{'='*60}")
    print(f"✅ TRAINING COMPLETED SUCCESSFULLY!")
    print(f"{'='*60}")
    print(f"⏱️ Total training time: {training_time/3600:.2f} hours")


    # Log training completion
    wandb.log({
        "training_completed": True,
        "total_training_time_hours": training_time / 3600,

        "curriculum_phases": 2,
        "final_epoch": NUM_EPOCHS
    })

except KeyboardInterrupt:
    print(f"\n⚠️ Training interrupted by user")
    print(f"💾 Last checkpoint should be saved in W&B artifacts")

except Exception as e:
    print(f"\n❌ Training failed with error: {e}")
    import traceback
    traceback.print_exc()

    # Log error
    wandb.log({"training_error": str(e)})

finally:
    # Final memory cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f"\n🧹 Memory cleanup completed")

🏋️ Starting QLoRA training with curriculum learning...
🎯 Target: Improve Answer F1 score on HotpotQA multihop reasoning
⏱️ Estimated time: 2.5+ hours

🚀 TRAINING STARTED - Monitor at: https://wandb.ai/jeffgong11235/hotpotqa-qlora/runs/eod1tqyc

📚 PHASE 1: Curriculum Learning (forced gold passages)
   Gold context rate: 100.00%

--- Example 1 (HotpotQADataCollator) ---
--- Example 1 (HotpotQADataCollator) ---

  Full Text (first 400 chars): <s>[INST]  

Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its possible that some of the evidences are irrelevant to the question and answer could not find enough sources to support.
 Respond with the answer directly and cite indices like [1], [3]([1] refers to the first evidence provided to you). If the an answer could not be reasoned th...  Full Text (first 400 chars): <s>[INST]  

Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its

Step,Training Loss
50,0.562200
100,0.047300


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

📦 Adapter zip created: ./qlora-checkpoints/checkpoint-125.zip (221.1 MB)
📤 Uploaded artifact with aliases: ['latest']

💾 Saving checkpoint after Phase 1...
📦 Adapter zip created: ./qlora-checkpoints/checkpoint-125.zip (148.1 MB)
📤 Uploaded artifact with aliases: ['latest']
✅ Checkpoint saved after Phase 1.

🎯 PHASE 2: Realistic Training (gold may be missing)
   Gold context rate: 100.00%
Continuing for 1 more epochs...

🏋️ Starting Phase 2, Epoch 2/2

--- Example 1 (HotpotQADataCollator) ---
--- Example 1 (HotpotQADataCollator) ---

  Full Text (first 400 chars): <s>[INST]  

Answer concisely by performing reasoning ONLY with selected sources from the evidences provided with you. Its possible that some of the evidences are irrelevant to the question and answer could not find enough sources to support.
 Respond with the answer directly and cite indices like [1], [3]([1] refers to the first evidence provided to you). If the an answer could not be reasoned th...  Full Text (first 400 char

Step,Training Loss
50,0.073100
100,0.025400


📦 Adapter zip created: ./qlora-checkpoints/checkpoint-125.zip (220.9 MB)
📤 Uploaded artifact with aliases: ['latest']

✅ TRAINING COMPLETED SUCCESSFULLY!
⏱️ Total training time: 1.30 hours

🧹 Memory cleanup completed


##  Fine-tuned Model Evaluation

This section evaluates the QLoRA fine-tuned model after training.

In [ ]:
# --- Load Fine-tuned Model from W&B Artifact ---
print("📊 Loading fine-tuned model from W&B artifact for evaluation...")

# --- Define bnb_config here to make the cell more self-contained ---
# This is needed to load the base quantized model
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)
# ------------------------------------------------------------------

eval_model = None
base_model_for_eval = None
adapter_path = None

try:
    # Ensure W&B run object is available
    if 'run' not in globals() or run is None:
        print("❌ W&B run object not found. Cannot download artifact.")
        raise RuntimeError("W&B run object not initialized.")

    # Download the 'latest' model artifact from W&B
    print("\n📥 Downloading the 'latest' model artifact from W&B...")
    artifact = run.use_artifact(f"qlora-adapters:latest")
    print(artifact.metadata)
    artifact_dir = artifact.download() # Downloads the artifact contents (including the zip)
    print(f"✅ Artifact downloaded to: {artifact_dir}")

    # --- Find and extract the zip file ---
    import zipfile
    import os

    zip_files = [f for f in os.listdir(artifact_dir) if f.endswith('.zip')]

    if not zip_files:
         print("❌ No zip file found in the artifact.")
         raise FileNotFoundError("Adapter zip file not found in the downloaded artifact.")

    zip_path = os.path.join(artifact_dir, zip_files[0])
    # Extract to a subdirectory within the artifact download directory
    # Use a more robust extraction dir name based on zip filename
    extract_dir_name = os.path.splitext(zip_files[0])[0] + "_extracted"
    extract_dir = os.path.join(artifact_dir, extract_dir_name)
    os.makedirs(extract_dir, exist_ok=True)
    print(f"Attempting to extract {zip_path} to {extract_dir}")

    with zipfile.ZipFile(zip_path, 'r') as zipf:
        zipf.extractall(extract_dir)

    print(f"✅ Successfully extracted adapter files to {extract_dir}.")
    # Now the adapter path is the extracted directory
    adapter_path = extract_dir

    # --- DEBUG: List contents of extracted directory ---
    print(f"\n🔍 Contents of extracted adapter directory ({adapter_path}):")
    if os.path.exists(adapter_path):
        extracted_contents = os.listdir(adapter_path)
        if extracted_contents:
            for item in extracted_contents:
                item_path = os.path.join(adapter_path, item)
                item_type = "Dir" if os.path.isdir(item_path) else "File"
                try:
                    item_size = os.path.getsize(item_path) / 1024 / 1024 # Size in MB
                    print(f"- {item} ({item_type}, {item_size:.2f} MB)")
                except Exception as size_e:
                    print(f"- {item} ({item_type}, Error getting size: {size_e})")
        else:
            print("The extracted directory is empty.")
    else:
        print("Extracted directory not found.")
    print("-" * 40)
    # --- END DEBUG ---


    # Load the base model
    print("🔄 Loading base Mistral model for evaluation...")
    if baseline_model not in globals() or baseline_model is None:
      base_model_for_eval = AutoModelForCausalLM.from_pretrained(
          MODEL_NAME, # Ensure MODEL_NAME is defined
          quantization_config=bnb_config,
          device_map="auto",
          torch_dtype=torch.bfloat16, # Ensure torch is imported
          cache_dir=CACHE_DIR,        # Ensure CACHE_DIR is defined
          trust_remote_code=True,
          use_cache=False # Important for evaluation
      )
    else:
        base_model_for_eval = baseline_model
    print("✅ Base model loaded.")

    # Load the PEFT adapter onto the base model
    print(f"\n🔧 Attempting to load adapter from path: {adapter_path}")
    from peft import PeftModel # Ensure PeftModel is imported
    eval_model = PeftModel.from_pretrained(base_model_for_eval, adapter_path)
    print("✅ Successfully loaded fine-tuned model from W&B artifact.")
    eval_model.eval() # Set the model to evaluation mode
    print("✅ Set eval_model to evaluation mode.")


except Exception as e:
    print(f"❌ Failed to load fine-tuned model from W&B artifact: {e}")
    import traceback
    traceback.print_exc()
#     # Clean up loaded components if any
#     if 'eval_model' in locals() and eval_model is not None: del eval_model
#     if 'base_model_for_eval' in locals() and base_model_for_eval is not None: del base_model_for_eval
#     if torch.cuda.is_available(): torch.cuda.empty_cache()
#     raise RuntimeError("Failed to load fine-tuned model for evaluation.") from e

# print("\n✅ Fine-tuned model loaded successfully as 'eval_model'!")
# print("📝 Next step: Implement the evaluation loop using 'eval_model'.")




# The rest of the evaluation logic will go into a subsequent cell based on the plan.
# This cell is ONLY for loading the model.

📊 Loading fine-tuned model from W&B artifact for evaluation...

📥 Downloading the 'latest' model artifact from W&B...
{'step': 125, 'epoch': 1.0, 'base_model': 'mistralai/Mistral-7B-Instruct-v0.2', 'train_loss': 0, 'learning_rate': 9.9e-05}


wandb: Downloading large artifact 'qlora-adapters:latest', 220.95MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:15.4 (14.4MB/s)


✅ Artifact downloaded to: /content/artifacts/qlora-adapters:v37
Attempting to extract /content/artifacts/qlora-adapters:v37/checkpoint-125.zip to /content/artifacts/qlora-adapters:v37/checkpoint-125_extracted
✅ Successfully extracted adapter files to /content/artifacts/qlora-adapters:v37/checkpoint-125_extracted.

🔍 Contents of extracted adapter directory (/content/artifacts/qlora-adapters:v37/checkpoint-125_extracted):
- trainer_state.json (File, 0.00 MB)
- tokenizer.json (File, 3.34 MB)
- training_args.bin (File, 0.01 MB)
- adapter_config.json (File, 0.00 MB)
- special_tokens_map.json (File, 0.00 MB)
- scheduler.pt (File, 0.00 MB)
- scaler.pt (File, 0.00 MB)
- tokenizer.model (File, 0.47 MB)
- tokenizer_config.json (File, 0.00 MB)
- chat_template.jinja (File, 0.00 MB)
- optimizer.pt (File, 81.76 MB)
- README.md (File, 0.00 MB)
- rng_state.pth (File, 0.01 MB)
- adapter_model.safetensors (File, 160.06 MB)
----------------------------------------
🔄 Loading base Mistral model for evaluat

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Base model loaded.

🔧 Attempting to load adapter from path: /content/artifacts/qlora-adapters:v37/checkpoint-125_extracted
✅ Successfully loaded fine-tuned model from W&B artifact.
✅ Set eval_model to evaluation mode.


### Inference Demo & Sanity Check

Quick inference on sample examples to verify model behavior.

In [ ]:
#sanity check with Example chat with the fine-tuned model
print("\n💬 Chatting with the fine-tuned model:")

# Define a simple prompt
chat_prompt = "What is the capital of France?"

# Tokenize the prompt
inputs = tokenizer(
    chat_prompt,
    return_tensors="pt",
    truncation=True,
    max_length=MAX_SEQ_LENGTH - 100 # Ensure space for generation
).to(eval_model.device)

# Generate a response
with torch.no_grad():
    outputs = eval_model.generate(
        **inputs,
        max_new_tokens=50, # Generate up to 50 new tokens
        temperature=0.7, # Add some randomness
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

# Decode the generated response
# Decode only the new tokens generated by the model
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

print(f"Prompt: {chat_prompt}")
print(f"Response: {response}")


💬 Chatting with the fine-tuned model:
Prompt: What is the capital of France?
Response: 

Paris

The capital city of France is Paris. Paris is known as the "City of Light," as well as the "City of Love." It's a major European cultural, economic, and political center, and home to


In [ ]:
# Fine-tuned Model Inference Demo for Debugging and Visualization
print("🧪 FINE-TUNED MODEL INFERENCE DEMO: Debugging and Visualization")
print("=" * 80)

# Ensure necessary variables and functions are defined
if 'eval_dataset' not in globals() or eval_dataset is None:
    raise RuntimeError("eval_dataset is not loaded. Please run the data loading cells first.")
if 'eval_model' not in globals() or eval_model is None:
    raise RuntimeError("eval_model is not loaded. Please run the cell to load the fine-tuned model first.")
else:
    eval_model.eval() # Ensure fine-tuned model is in eval mode
    print("✅ Using loaded fine-tuned model ('eval_model') for demo.")


if 'evaluator' not in globals() or evaluator is None:
    raise RuntimeError("HotpotQAEvaluator is not initialized. Please run the evaluation setup cell.")
if 'generate_answer' not in globals():
    raise RuntimeError("generate_answer function is not defined. Please run the evaluation setup cell.")
if 'extract_answer_and_citations' not in globals():
    raise RuntimeError("extract_answer_and_citations function is not defined. Please run the evaluation setup cell.")
if 'building_prompts_rag' not in globals():
     print("⚠️ building_prompts_rag not found. Using default RAG instruction (might affect quality).")
     building_prompts_rag = {'instruction': "Answer the question using the provided evidence.", 'cot_exemplar': ""}
# MODEL_NAME, bnb_config, CACHE_DIR are not needed in this cell anymore as we are not loading the base model here.


print(f"\n📊 Evaluation dataset size: {len(eval_dataset)}")

# --- Reduce number of examples and max_new_tokens for faster debugging ---
num_examples = min(20, len(eval_dataset)) # Reduced to 2 examples
temp_max_new_tokens = 300 # Reduced max new tokens
print(f"📝 Testing on {num_examples} examples with max_new_tokens={temp_max_new_tokens}...")
# --- End Reduction ---


# Select a few examples from the evaluation dataset for the demo
demo_examples = eval_dataset.shuffle(seed=42).select(range(num_examples))


for i, example in enumerate(demo_examples):
    print(f"\n" + "="*100)
    print(f"📝 EXAMPLE {i+1}: Fine-tuned Model Prediction")
    print(f"="*100)
    question = example['question']
    gold_answer_text = example['answer']
    passages = example['passages']

    print(f"❓ Question: {question}")
    print(f"✅ Gold Answer: {gold_answer_text}")

    print(f"\n📚 Available Evidence Passages (first 3 titles & snippets):")
    for j, passage in enumerate(passages[:3], 1):
        print(f"   [{j}] {passage.get('title', 'N/A')}: {passage.get('text', '')[:100]}...")
    if len(passages) > 3:
         print(f"   ...and {len(passages)-3} more passages.")


    print(f"\n🤖 FINE-TUNED MODEL PREDICTION:")
    print(f"{'='*60}")

    try:
        # Generate prediction using the fine-tuned eval_model
        # Ensure generate_answer uses building_prompts_rag for the fine-tuned model
        # Use the reduced max_new_tokens for this demo
        finetuned_prediction = generate_answer(question, passages, building_prompts_rag, eval_model, max_new_tokens=temp_max_new_tokens)
        print(f"   {finetuned_prediction}")

        # Extract answers and citations
        finetuned_answer, finetuned_citations = extract_answer_and_citations(finetuned_prediction)
        gold_answer, gold_citations = extract_answer_and_citations(gold_answer_text)

        # Calculate metrics
        finetuned_f1 = evaluator.answer_f1_score(finetuned_answer, gold_answer)
        finetuned_em = evaluator.answer_exact_match(finetuned_answer, gold_answer)
        finetuned_citation_acc = evaluator.answer_f1_score(str(finetuned_citations), str(gold_citations)) # Simple citation F1 on string repr

        print(f"\n   Metrics - F1: {finetuned_f1:.3f} | EM: {finetuned_em:.3f} | Citations: {finetuned_citations} (Gold: {gold_citations})")
    except Exception as e:
         print(f"   ❌ Error generating fine-tuned prediction: {e}")
         import traceback
         traceback.print_exc()
         finetuned_prediction = "Error generating prediction."
         finetuned_f1, finetuned_em, finetuned_citation_acc = 0.0, 0.0, 0.0
         print(f"\n   Metrics - F1: {finetuned_f1:.3f} | EM: {finetuned_em:.3f} | Citations: N/A (Gold: {gold_citations if 'gold_citations' in locals() else 'N/A'})")


    print("-" * 100)


# No cleanup needed for eval_model here, as it's loaded in a separate cell.
# Cleanup CUDA memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("\n🧹 CUDA memory cleared after demo.")

print(f"\n{'='*80}")
print(f"✅ FINE-TUNED MODEL INFERENCE DEMO COMPLETE!")
print(f"{'='*80}")

🧪 FINE-TUNED MODEL INFERENCE DEMO: Debugging and Visualization
✅ Using loaded fine-tuned model ('eval_model') for demo.

📊 Evaluation dataset size: 200
📝 Testing on 20 examples with max_new_tokens=300...

📝 EXAMPLE 1: Fine-tuned Model Prediction
❓ Question: Bangalore Naatkal starred which actor and photographer?
✅ Gold Answer: {
  "reasoning": "Relevant evidence found in passages [2], [4].",
  "answer": "Ramanaidu Daggubati",
  "citations": [
    2,
    4
  ]
}

📚 Available Evidence Passages (first 3 titles & snippets):
   [1] Antha Ezhu Naatkal: Antha 7 Naatkal (read as ""Antha Ezhu Naatkal""; English: "Those Seven Days" ) is a 1981 Tamil langu...
   [2] Bangalore Naatkal: Bangalore Naatkal (English: "Bangalore Days" ) is a 2016 Indian Tamil comedy-drama film directed by ...
   [3] Saad Khan: Saad Khan, born in Mumbai, India, is an Indian film director, screenwriter, acting teacher, founder ...
   ...and 5 more passages.

🤖 FINE-TUNED MODEL PREDICTION:
Prompt length: 6218
the maximum 

As we could see that the finetuning result of LLM make it stupid and replies mechanically. The debugging implies problem is with the Lora Adapter. The issue usualy has to do with data& format, such as validation data preparation is problematic or loss computation is undesired from ground-truth or prompt template formatting has issue.

### Fine-tuned Model Full Evaluation

Comprehensive evaluation of fine-tuned model on full evaluation dataset.

In [ ]:
# Fine-tuned Model Full Evaluation
# Evaluate QLoRA fine-tuned model on full evaluation dataset

print("🚀 Starting fine-tuned model evaluation...")
print(f"   Model: Mistral-7B-Instruct (QLoRA fine-tuned)")
print(f"   Strategy: Direct JSON output (instruction-tuned)")
print(f"   Dataset: Full eval_dataset ({len(eval_dataset)} examples)\n")

# Evaluate using unified function
finetuned_results = evaluate_model_comprehensive(
    model=eval_model,
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
    evaluator=evaluator,
    model_name="Fine-tuned QLoRA",
    max_examples=None,  # Evaluate on full dataset
    use_rag_prompting=False,  # Use direct input_text from dataset
    verbose_level="sample",  # Print first 5 examples
    wandb_prefix="final_eval",
    building_prompts=None
)

# Store for later comparison
print("✅ Fine-tuned evaluation complete!")
print(f"📊 Key Results:")
print(f"   • Exact Match: {finetuned_results['em']:.1%}")
print(f"   • F1 Score: {finetuned_results['f1']:.3f}")
print(f"   • Citation F1: {finetuned_results['citation_f1']:.3f}")
print(f"   • Insufficient Context Detection: {finetuned_results['insufficient_context_rate']:.1%}")


🚀 Starting fine-tuned model evaluation...
   Model: Mistral-7B-Instruct (QLoRA fine-tuned)
   Strategy: Direct JSON output (instruction-tuned)
   Dataset: Full eval_dataset (200 examples)


🔍 Evaluating Fine-tuned QLoRA on 200 examples...



Evaluating Fine-tuned QLoRA:   0%|          | 0/200 [00:00<?, ?it/s]


--- Example 1 ---
Question: What nationality was Oliver Reed's character in the film Royal Flash?...
[Ex 0] 🔍 Extracting from:  {   "reasoning": "Based on the available evidence, I cannot determine a definitive answer to this question.",   "answer": "insufficient context",   "citations": [] }...
[Ex 0] ✅ JSON parse OK: answer='insufficient context...', citations=[]
[Ex 0] 🔍 Extracting from: {   "reasoning": "Based on the available evidence, I cannot determine a definitive answer to this question.",   "answer": "insufficient context",   "citations": [] }...
[Ex 0] ✅ JSON parse OK: answer='insufficient context...', citations=[]
Predicted: insufficient context
Gold: insufficient context
Pred Citations: []
Gold Citations: []
F1: 1.000, EM: 1.000, Citation F1: 1.000


Evaluating Fine-tuned QLoRA:   0%|          | 1/200 [00:11<38:31, 11.61s/it]


--- Example 2 ---
Question: Pacific Mozart Ensemble performed which German composer's Der Lindberghflug in 2002?...
[Ex 1] 🔍 Extracting from: in 1874 in Munich.  The opera is based on the eponymous play by Shakespeare, which itself is based on a much older English folk ballad.  It is a well-known fact that the work is based on the English p...
[Ex 1] ⚠️  JSON parse failed: Expecting value: line 1 column 1 (char 0). Trying fallback...
[Ex 1] 🔄 Using regex fallback...
[Ex 1] 📝 Regex result: answer='in 1874 in Munich.  The opera is based on the epon...', citations=[]
[Ex 1] 🔍 Extracting from: {   "reasoning": "From evidence [7]: Peter Wallace Hobbs formed the electrical appliance company Russell Hobbs with Bill Russell From evidence [8]: Russell Hobbs is a manufacturer of household applian...
[Ex 1] ✅ JSON parse OK: answer='Kurt Julian Weill...', citations=[5, 8]
Predicted: in 1874 in Munich.  The opera is based on the eponymous play by Shakespeare, which itself is based o...
Gold: Kurt 

...

[Ex 198] 🔍 Extracting from:  {   "reasoning": "To answer this question, evidence [3] shows that Brian Harold Billick (born February 28, 1954) is a former National Football League coach and commentator., and evidence [4] indicate...
[Ex 198] ✅ JSON parse OK: answer='the third season...', citations=[3, 4]
[Ex 198] 🔍 Extracting from: {   "reasoning": "Relevant evidence found in passages [3], [4].",   "answer": "6th season",   "citations": [     3,     4   ] }...
[Ex 198] ✅ JSON parse OK: answer='6th season...', citations=[3, 4]


Evaluating Fine-tuned QLoRA: 100%|█████████▉| 199/200 [2:05:54<00:35, 35.79s/it]

[Ex 199] 🔍 Extracting from:  {   "reasoning": "Based on the available evidence, I cannot determine a definitive answer to this question.",   "answer": "insufficient context",   "citations": [] }  This is because Vocelli Pizza an...
[Ex 199] ⚠️  JSON parse failed: Extra data: line 8 column 1 (char 168). Trying fallback...
[Ex 199] 🔍 Found JSON substring, parsing...
[Ex 199] ✅ Substring parse OK: answer='insufficient context...', citations=[]
[Ex 199] 🔍 Extracting from: {   "reasoning": "Relevant evidence found in passages [1], [3].",   "answer": "Pizza",   "citations": [     1,     3   ] }...
[Ex 199] ✅ JSON parse OK: answer='Pizza...', citations=[1, 3]


Evaluating Fine-tuned QLoRA: 100%|██████████| 200/200 [2:07:08<00:00, 38.14s/it]


📊 FINE-TUNED QLORA - EVALUATION RESULTS
Total Examples: 200
Exact Match (EM): 0.415
F1 Score: 0.464
Citation Precision: 0.575
Citation Recall: 0.750
Citation F1: 0.575
Insufficient Context Detection: 60.0% (51/85)

✅ Fine-tuned evaluation complete!
📊 Key Results:
   • Exact Match: 41.5%
   • F1 Score: 0.464
   • Citation F1: 0.575
   • Insufficient Context Detection: 60.0%


##  Baseline vs Fine-tuned Comparison

Side-by-side comparison of baseline RAG prompting approach vs QLoRA fine-tuned model.

In [3]:

# Comprehensive Side-by-Side Comparison
import pandas as pd

print("\n" + "="*80)
print("📊 BASELINE vs FINE-TUNED MODEL COMPARISON")
print("="*80 + "\n")

baseline_results={'em':0.175, 'f1':0.274, 'citation_f1':0.402, 'citation_precision':0.458,'citation_recall':0.703 ,'insufficient_context_detection_rate':0.117, 'total_examples':200}


finetuned_results = {'em':0.415,'f1':0.464,'citation_f1':0.575, 'citation_precision':0.575, 'citation_recall':0.750, 'insufficient_context_detection_rate':0.6, 'total_examples':200}



# Create comparison DataFrame
comparison_data = {
    'Metric': [
        'Exact Match (EM)',
        'F1 Score',
        'Citation Precision',
        'Citation Recall',
        'Citation F1',
        'Insufficient Context Detection'
    ],
    'Baseline (RAG)': [
        baseline_results['em'],
        baseline_results['f1'],
        baseline_results['citation_precision'],
        baseline_results['citation_recall'],
        baseline_results['citation_f1'],
        baseline_results['insufficient_context_detection_rate']
    ],
    'Fine-tuned (QLoRA)': [
        finetuned_results['em'],
        finetuned_results['f1'],
        finetuned_results['citation_precision'],
        finetuned_results['citation_recall'],
        finetuned_results['citation_f1'],
        finetuned_results['insufficient_context_detection_rate']
    ]
}

comparison_df = pd.DataFrame(comparison_data)

# Calculate improvements
comparison_df['Δ (Absolute)'] = comparison_df['Fine-tuned (QLoRA)'] - comparison_df['Baseline (RAG)']
comparison_df['Δ (%)'] = (comparison_df['Δ (Absolute)'] / comparison_df['Baseline (RAG)']) * 100

# Format for display
comparison_df_display = comparison_df.copy()
comparison_df_display['Baseline (RAG)'] = comparison_df_display['Baseline (RAG)'].apply(lambda x: f"{x:.3f}")
comparison_df_display['Fine-tuned (QLoRA)'] = comparison_df_display['Fine-tuned (QLoRA)'].apply(lambda x: f"{x:.3f}")
comparison_df_display['Δ (Absolute)'] = comparison_df_display['Δ (Absolute)'].apply(lambda x: f"{x:+.3f}")
comparison_df_display['Δ (%)'] = comparison_df_display['Δ (%)'].apply(lambda x: f"{x:+.1f}%")

print(comparison_df_display.to_string(index=False))
print("\n" + "="*80)

# Summary statistics
print("\n📈 SUMMARY:")
print(f"   • Dataset sizes: Baseline (100 examples) vs Fine-tuned ({finetuned_results['total_examples']} examples)")
print(f"   • Average improvement: {comparison_df['Δ (Absolute)'].mean():.3f} ({comparison_df['Δ (%)'].mean():+.1f}%)")

# Identify best improvements
best_metric = comparison_df.loc[comparison_df['Δ (Absolute)'].idxmax(), 'Metric']
best_improvement = comparison_df.loc[comparison_df['Δ (Absolute)'].idxmax(), 'Δ (Absolute)']
best_improvement_pct = comparison_df.loc[comparison_df['Δ (Absolute)'].idxmax(), 'Δ (%)']
print(f"   • Best improvement: {best_metric} (+{best_improvement:.3f}, +{best_improvement_pct:.1f}%)")



# # Log comparison to W&B
# if wandb.run:
#     # Create W&B table
#     comparison_table = wandb.Table(dataframe=comparison_df)
#     wandb.log({
#         "model_comparison": comparison_table,
#         "avg_improvement_absolute": comparison_df['Δ (Absolute)'].mean(),
#         "avg_improvement_percent": comparison_df['Δ (%)'].mean()
#     })

#     # Log bar chart comparison
#     metrics_for_chart = ['Exact Match (EM)', 'F1 Score', 'Citation F1']
#     chart_data = []
#     for metric in metrics_for_chart:
#         row = comparison_df[comparison_df['Metric'] == metric].iloc[0]
#         chart_data.append([metric, "Baseline", row['Baseline (RAG)']])
#         chart_data.append([metric, "Fine-tuned", row['Fine-tuned (QLoRA)']])

#     chart_table = wandb.Table(data=chart_data, columns=["Metric", "Model", "Score"])
#     wandb.log({
#         "comparison_bar_chart": wandb.plot.bar(
#             chart_table,
#             "Metric",
#             "Score",
#             title="Baseline vs Fine-tuned Performance"
#         )
#     })

    # print("\n✅ Comparison logged to W&B!")

print("\n" + "="*80)
print("🎉 Evaluation complete! Fine-tuned model shows improvement across all metrics.")
print("="*80)



📊 BASELINE vs FINE-TUNED MODEL COMPARISON

                        Metric Baseline (RAG) Fine-tuned (QLoRA) Δ (Absolute)   Δ (%)
              Exact Match (EM)          0.175              0.415       +0.240 +137.1%
                      F1 Score          0.274              0.464       +0.190  +69.3%
            Citation Precision          0.458              0.575       +0.117  +25.5%
               Citation Recall          0.703              0.750       +0.047   +6.7%
                   Citation F1          0.402              0.575       +0.173  +43.0%
Insufficient Context Detection          0.117              0.600       +0.483 +412.8%


📈 SUMMARY:
   • Dataset sizes: Baseline (100 examples) vs Fine-tuned (200 examples)
   • Average improvement: 0.208 (+115.8%)
   • Best improvement: Insufficient Context Detection (+0.483, +412.8%)

🎉 Evaluation complete! Fine-tuned model shows improvement across all metrics.
